<a href="https://colab.research.google.com/github/littlericecat/test/blob/main/Quant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install ccxt pandas


In [5]:
!pip install backtesting

In [3]:
import ccxt
import pandas as pd
import time
import datetime
import os

# --- Configuration ---
EXCHANGE_ID = 'binanceus'  # Example: 'binance', 'kraken', 'coinbasepro', 'bybit'
SYMBOLS_TO_FETCH = ['BTC/USDT', 'ETH/USDT', 'SOL/USDT']
TIMEFRAMES_TO_FETCH = ['1d', '1h', '15m'] # Daily, Hourly, 15-minute

# Calculate date range (4 years from today, May 14, 2025)
# Current date is May 14, 2025
end_date_dt = datetime.datetime(2025, 5, 14, 0, 0, 0, tzinfo=datetime.timezone.utc)
start_date_dt = end_date_dt - datetime.timedelta(days=4*365)

# Convert to milliseconds timestamps as required by CCXT
since_timestamp = int(start_date_dt.timestamp() * 1000)
to_timestamp = int(end_date_dt.timestamp() * 1000)

# --- MODIFICATION FOR COLAB: Set Output Directory to Google Drive ---
# Ensure you have mounted your Google Drive in the previous step
# Create a folder named 'CryptoDataCCXT' in your Google Drive's root if it doesn't exist
DRIVE_OUTPUT_PATH = '/content/drive/MyDrive/CryptoDataCCXT' # Change if you want a different folder
if not os.path.exists(DRIVE_OUTPUT_PATH):
    os.makedirs(DRIVE_OUTPUT_PATH)
OUTPUT_DIRECTORY = DRIVE_OUTPUT_PATH
# If you don't want to use Google Drive (data will be temporary):
# OUTPUT_DIRECTORY = "crypto_ohlcv_data"
# if not os.path.exists(OUTPUT_DIRECTORY):
#     os.makedirs(OUTPUT_DIRECTORY)

print(f"Data will be saved to: {OUTPUT_DIRECTORY}")

# --- Initialize Exchange ---
exchange_options = {
    'options': {
        'adjustForTimeDifference': True,
    },
    # 'enableRateLimit': True, # Consider enabling for CCXT's own rate limiting
}
exchange = getattr(ccxt, EXCHANGE_ID)(exchange_options)

# --- Helper Function to Fetch All OHLCV Data ---
def fetch_all_ohlcv(symbol, timeframe, since, limit=1000):
    all_candles = []
    current_since = since
    now = exchange.milliseconds() # To avoid fetching beyond current time if not needed
                                 # Though we are using a fixed 'to_timestamp' for this 4-year historical pull

    print(f"Starting fetch for {symbol} ({timeframe}) from {exchange.iso8601(current_since)} up to {exchange.iso8601(to_timestamp)}")

    while current_since < to_timestamp: # Ensure we don't fetch beyond our defined 4-year end period
        try:
            # Conservative delay before each request
            # For 15m data over 4 years, this delay is CRUCIAL.
            # Binance limit is ~1200 weight/min. fetchOHLCV might be 1-10 weight.
            # 0.5s to 1s is a common starting point for non-critical scripts.
            # Increase if you hit rate limit errors (429, 418).
            time.sleep(exchange.rateLimit / 1000 + 0.75) # Adjusted delay

            candles = exchange.fetch_ohlcv(symbol, timeframe, since=current_since, limit=limit)

            if candles and len(candles) > 0:
                all_candles.extend(candles)
                last_timestamp_fetched = candles[-1][0]
                print(f"Fetched {len(candles)} candles for {symbol} ({timeframe}) up to {exchange.iso8601(last_timestamp_fetched)}")

                if last_timestamp_fetched >= current_since:
                     # Advance 'since' to the timestamp of the candle AFTER the last one received
                    current_since = last_timestamp_fetched + (exchange.parse_timeframe(timeframe) * 1000)
                else:
                    print(f"Warning: Timestamp did not advance for {symbol} ({timeframe}). Current 'since': {exchange.iso8601(current_since)}, last fetched: {exchange.iso8601(last_timestamp_fetched)}. Breaking.")
                    break

                # Stop if the next 'since' would be past our overall desired end period
                if current_since > to_timestamp:
                    print(f"Next fetch for {symbol} ({timeframe}) would start after target end date {exchange.iso8601(to_timestamp)}. Current 'since': {exchange.iso8601(current_since)}. Stopping current timeframe fetch.")
                    break
            else:
                print(f"No more data for {symbol} ({timeframe}) after {exchange.iso8601(current_since)}. Moving to next timeframe or symbol.")
                break # No more data for this specific symbol/timeframe starting from current_since

        except ccxt.NetworkError as e:
            print(f"NetworkError fetching {symbol} ({timeframe}): {e}. Retrying in 30s...")
            time.sleep(30)
        except ccxt.DDoSProtection as e: # Specifically handle DDoS protection errors
            print(f"DDoSProtection/RateLimit fetching {symbol} ({timeframe}): {e}. Retrying in 120s...")
            time.sleep(120)
        except ccxt.RateLimitExceeded as e: # Specifically handle RateLimitExceeded errors
            print(f"RateLimitExceeded fetching {symbol} ({timeframe}): {e}. Retrying in 120s...")
            time.sleep(120)
        except ccxt.ExchangeError as e: # Other exchange errors
            print(f"ExchangeError fetching {symbol} ({timeframe}): {e}. Retrying in 60s...")
            time.sleep(60)
        except Exception as e: # Other unexpected errors
            print(f"An unexpected error occurred fetching {symbol} ({timeframe}): {e}. Retrying in 60s...")
            time.sleep(60)

    # Post-loop processing for collected candles for the current symbol/timeframe
    if all_candles:
        df_temp = pd.DataFrame(all_candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df_temp.drop_duplicates(subset=['timestamp'], keep='first', inplace=True)
        # Filter out any data that might have been fetched beyond our strict 'to_timestamp'
        # This can happen if the last chunk fetched candles that crossed the 'to_timestamp'
        df_temp = df_temp[df_temp['timestamp'] < to_timestamp] # strictly less than, as 'to_timestamp' is the start of the next interval
        all_candles = df_temp.values.tolist()

    return all_candles

# --- Main Loop ---
for symbol in SYMBOLS_TO_FETCH:
    print(f"\n--- Processing Symbol: {symbol} ---")
    for timeframe in TIMEFRAMES_TO_FETCH:
        print(f"  Fetching timeframe: {timeframe}")

        safe_symbol_name = symbol.replace("/", "_")
        filename = os.path.join(OUTPUT_DIRECTORY, f"{safe_symbol_name}_{timeframe}_4years.csv")

        if os.path.exists(filename):
            print(f"  Data for {symbol} ({timeframe}) already exists: {filename}. Skipping.")
            continue

        fetch_limit = 1000 # Default limit
        # (You can adjust limit per timeframe/exchange if needed, but 1000 is common for Binance)

        ohlcv_data = fetch_all_ohlcv(symbol, timeframe, since_timestamp, limit=fetch_limit)

        if ohlcv_data:
            df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['datetime_utc'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
            # Ensure the datetime_utc is also before the 'to_timestamp'
            df = df[df['timestamp'] < to_timestamp]
            df = df[['datetime_utc', 'open', 'high', 'low', 'close', 'volume', 'timestamp']]

            if not df.empty:
                df.to_csv(filename, index=False)
                print(f"  Successfully saved data to {filename}")
                print(f"  Fetched {len(df)} records for {symbol} ({timeframe}).")
                if not df.empty:
                    print(f"  Data ranges from {df['datetime_utc'].min()} to {df['datetime_utc'].max()}")
            else:
                 print(f"  No data within the specified date range for {symbol} ({timeframe}) after filtering. File not saved.")
        else:
            print(f"  No data fetched for {symbol} ({timeframe}).")

print("\n--- All Done! ---")

Data will be saved to: /content/drive/MyDrive/CryptoDataCCXT

--- Processing Symbol: BTC/USDT ---
  Fetching timeframe: 1d
Starting fetch for BTC/USDT (1d) from 2021-05-15T00:00:00.000Z up to 2025-05-14T00:00:00.000Z
Fetched 1000 candles for BTC/USDT (1d) up to 2024-02-08T00:00:00.000Z
Fetched 476 candles for BTC/USDT (1d) up to 2025-05-29T00:00:00.000Z
Next fetch for BTC/USDT (1d) would start after target end date 2025-05-14T00:00:00.000Z. Current 'since': 2025-05-30T00:00:00.000Z. Stopping current timeframe fetch.
  Successfully saved data to /content/drive/MyDrive/CryptoDataCCXT/BTC_USDT_1d_4years.csv
  Fetched 1460 records for BTC/USDT (1d).
  Data ranges from 2021-05-15 00:00:00+00:00 to 2025-05-13 00:00:00+00:00
  Fetching timeframe: 1h
Starting fetch for BTC/USDT (1h) from 2021-05-15T00:00:00.000Z up to 2025-05-14T00:00:00.000Z
Fetched 1000 candles for BTC/USDT (1h) up to 2021-06-25T22:00:00.000Z
Fetched 1000 candles for BTC/USDT (1h) up to 2021-08-06T14:00:00.000Z
Fetched 1000 

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import os

# --- Configuration ---

# Option 1: Where are the files you want to check?
# "colab_session" - for files in the current Colab session's temporary storage (likely in a subfolder)
# "google_drive" - for files already saved to your Google Drive
FILE_LOCATION_TYPE = "google_drive" # Or "colab_session"

# Define the symbols and timeframes from your CCXT script
SYMBOLS = ['BTC_USDT', 'ETH_USDT', 'SOL_USDT']
TIMEFRAMES = ['1d', '1h', '15m'] # CCXT format
DURATION_TAG = "4years" # From your CCXT script's naming

# Generate the list of expected CSV file names
EXPECTED_CSV_FILES = []
for symbol_base in SYMBOLS:
    for tf_ccxt in TIMEFRAMES:
        # Convert CCXT timeframe to a more descriptive tag if needed, or use as is
        # For simplicity, using CCXT tf_ccxt directly in filename part
        filename = f"{symbol_base}_{tf_ccxt}_{DURATION_TAG}.csv"
        EXPECTED_CSV_FILES.append(filename)

print("Expected files to check:")
for f_name in EXPECTED_CSV_FILES:
    print(f"- {f_name}")


# If checking from Google Drive, specify the base path (should match your CCXT script's output)
GDRIVE_BASE_PATH = "/content/drive/MyDrive/CryptoDataCCXT/" # Adjust if you used a different folder

# If checking from Colab session, specify the subfolder used by the CCXT script
DEFAULT_SESSION_SUBFOLDER = "crypto_ohlcv_data"

# --- Mount Google Drive if needed ---
if FILE_LOCATION_TYPE == "google_drive":
    from google.colab import drive
    try:
        drive.mount('/content/drive', force_remount=False) # force_remount=False is gentler
        print(f"Google Drive mounted. Checking files in: {GDRIVE_BASE_PATH}")
    except Exception as e:
        print(f"Could not mount Google Drive: {e}. Cannot check files from Drive.")
        # Clear the list to prevent errors trying to access GDrive paths if mount fails
        EXPECTED_CSV_FILES = []

# --- Loop Through Files and Check Format ---
print("\n--- Starting Format Check for All Specified CSV Files ---")

all_files_summary = {} # To store summary status

for file_name_to_check in EXPECTED_CSV_FILES:
    path_to_file = None
    if FILE_LOCATION_TYPE == "google_drive":
        path_to_file = os.path.join(GDRIVE_BASE_PATH, file_name_to_check)
    else: # colab_session
        # Assumes file is in the specified subfolder or /content/subfolder
        session_file_path = os.path.join(DEFAULT_SESSION_SUBFOLDER, file_name_to_check)
        if os.path.exists(session_file_path):
             path_to_file = session_file_path
        elif os.path.exists(f"/content/{session_file_path}"): # Check relative to /content/
             path_to_file = f"/content/{session_file_path}"
        elif os.path.exists(file_name_to_check): # Check current working directory (less likely for Colab)
            path_to_file = file_name_to_check

    print(f"\n\n======================================================================")
    print(f"--- Checking Format for: {file_name_to_check} ---")
    print(f"Attempting to locate at: {path_to_file if path_to_file else 'N/A (Path could not be determined for session check)'}")
    print(f"======================================================================")

    file_status = "ERROR: File not found or path undetermined."

    if path_to_file and os.path.exists(path_to_file):
        try:
            # Load the CSV. The CCXT script saved 'datetime_utc' as a string.
            # 'timestamp' (ms) was also saved.
            # We'll use 'datetime_utc' as the index.
            df = pd.read_csv(path_to_file, index_col='datetime_utc', parse_dates=True)
            file_status = "OK" # Initial status

            if df.empty:
                print("WARNING: The file is empty.")
                file_status = "WARNING: Empty file"
            else:
                print("\n1. First 5 rows:")
                print(df.head())

                print("\n2. Data types and non-null values:")
                df.info()

                print("\n3. Basic statistics for numerical columns:")
                print(df.describe())

                print("\n4. Index details:")
                print(f"   Index name: {df.index.name}")
                print(f"   Index type: {type(df.index)}")
                print(f"   Index data type: {df.index.dtype}") # Should be datetime64[ns, UTC]
                if not df.empty:
                    is_monotonic = df.index.is_monotonic_increasing or df.index.is_monotonic_decreasing # allow for either strictly
                    print(f"   Is index monotonic? {is_monotonic}")
                    if not is_monotonic:
                        file_status = "WARNING: Index not monotonic"
                    print(f"   Date range: {df.index.min()} to {df.index.max()}")
                print(f"   Number of rows: {len(df)}")

                # Basic Sanity Checks
                print("\n5. Column and Value Sanity Checks:")
                required_cols = ['open', 'high', 'low', 'close', 'volume']
                missing_cols = [col for col in required_cols if col not in df.columns]
                if missing_cols:
                    print(f"   WARNING: Missing expected OHLCV columns (lowercase): {', '.join(missing_cols)}")
                    file_status = f"WARNING: Missing columns ({', '.join(missing_cols)})"

                for col in ['open', 'high', 'low', 'close', 'volume']:
                    if col in df.columns and (df[col] < 0).any():
                        print(f"   WARNING: Negative values found in '{col}' column.")
                        file_status = f"WARNING: Negative values in {col}"

                # OHLC integrity checks
                if 'open' in df.columns and 'high' in df.columns and \
                   'low' in df.columns and 'close' in df.columns:
                    if not (df['high'] >= df['low']).all():
                        print("   WARNING: Not all 'high' values are >= 'low' values.")
                        file_status = "WARNING: High < Low inconsistency"
                    if not (df['high'] >= df['open']).all():
                        print("   WARNING: Not all 'high' values are >= 'open' values.")
                        file_status = "WARNING: High < Open inconsistency"
                    if not (df['high'] >= df['close']).all():
                        print("   WARNING: Not all 'high' values are >= 'close' values.")
                        file_status = "WARNING: High < Close inconsistency"
                    if not (df['low'] <= df['open']).all():
                        print("   WARNING: Not all 'low' values are <= 'open' values.")
                        file_status = "WARNING: Low > Open inconsistency"
                    if not (df['low'] <= df['close']).all():
                        print("   WARNING: Not all 'low' values are <= 'close' values.")
                        file_status = "WARNING: Low > Close inconsistency"
                else:
                    print("   Skipping OHLC integrity checks due to missing columns.")

        except pd.errors.EmptyDataError:
            error_message = f"ERROR: The file '{path_to_file}' is empty and cannot be parsed by Pandas."
            print(error_message)
            file_status = error_message
        except Exception as e:
            error_message = f"ERROR: An error occurred while reading or processing '{path_to_file}': {e}"
            print(error_message)
            file_status = error_message
    else:
        print(f"ERROR: File not found at the determined path: '{path_to_file if path_to_file else file_name_to_check}'. Skipping.")
        # file_status remains "ERROR: File not found..."

    all_files_summary[file_name_to_check] = file_status

print("\n\n--- Overall Summary of File Checks ---")
for file_name, status in all_files_summary.items():
    print(f"- {file_name}: {status}")
print("\n--- Format Check for All Files Complete ---")

In [6]:
# ==============================================================================
# --- Master Configuration for Indicator Parameters (Coin-Specific) ---
# ==============================================================================
# Define parameters for each coin. You can have different settings for BTC, ETH, SOL.
# If a coin is not listed here, or an indicator for a coin is not listed,
# you could fall back to a 'default' configuration or skip it.

master_indicator_configs = {
    "BTC": {
        "sma_short":    {"window": 10, "column": "Close"},
        "sma_long":     {"window": 50, "column": "Close"},
        "ema_short":    {"span": 12, "column": "Close"},
        "ema_long":     {"span": 26, "column": "Close"},
        "rsi":          {"window": 14, "column": "Close"},
        "bollinger":    {"window": 20, "num_std_dev": 2, "column": "Close"},
        "macd":         {"short_window": 12, "long_window": 26, "signal_window": 9, "column": "Close"},
        "atr":          {"window": 14},
        "stochastic":   {"k_window": 14, "d_window": 3},
        "mfi":          {"window": 14},
        "roc":          {"window": 10, "column": "Close"},
        "keltner":      {"ema_window": 20, "atr_window": 10, "atr_multiplier": 2},
        "volume_sma":   {"window": 20},
        "support_resistance": {"window": 15}
    },
    "ETH": {
        "sma_short":    {"window": 12, "column": "Close"}, # Slightly different SMA for ETH
        "sma_long":     {"window": 55, "column": "Close"},
        "ema_short":    {"span": 10, "column": "Close"},
        "ema_long":     {"span": 30, "column": "Close"},
        "rsi":          {"window": 14, "column": "Close"}, # Same RSI
        "bollinger":    {"window": 20, "num_std_dev": 2.1, "column": "Close"}, # Slightly wider BB
        "macd":         {"short_window": 12, "long_window": 26, "signal_window": 9, "column": "Close"},
        "atr":          {"window": 10},
        "stochastic":   {"k_window": 10, "d_window": 3},
        "mfi":          {"window": 12},
        "roc":          {"window": 12, "column": "Close"},
        "keltner":      {"ema_window": 22, "atr_window": 11, "atr_multiplier": 2},
        "volume_sma":   {"window": 25},
        "support_resistance": {"window": 20}
    },
    "SOL": {
        # Using mostly similar params to BTC for SOL as a starting point
        # You can fine-tune these specifically for SOL
        "sma_short":    {"window": 9, "column": "Close"},
        "sma_long":     {"window": 45, "column": "Close"},
        "ema_short":    {"span": 12, "column": "Close"},
        "ema_long":     {"span": 26, "column": "Close"},
        "rsi":          {"window": 14, "column": "Close"},
        "bollinger":    {"window": 20, "num_std_dev": 2, "column": "Close"},
        "macd":         {"short_window": 12, "long_window": 26, "signal_window": 9, "column": "Close"},
        "atr":          {"window": 14},
        "stochastic":   {"k_window": 14, "d_window": 3},
        "mfi":          {"window": 14},
        "roc":          {"window": 10, "column": "Close"},
        "keltner":      {"ema_window": 20, "atr_window": 10, "atr_multiplier": 1.8}, # Tighter Keltner for SOL
        "volume_sma":   {"window": 20},
        "support_resistance": {"window": 15}
    }
    # You can add a "DEFAULT" key here if you want to process coins not explicitly listed
}


https://drive.google.com/drive/u/0/search?q=parent:1sbKi08-dsL5C4FRpHgmM_RQbx9ldG0Ie

In [ ]:
# ==============================================================================
# --- Technical Indicator Functions (Paste the functions from previous response here) ---
# e.g., add_sma, add_ema, add_rsi, add_bollinger_bands, add_macd,
#       add_atr, add_stochastic_oscillator, add_mfi, add_roc,
#       add_keltner_channels, add_volume_sma, add_basic_support_resistance
# ==============================================================================
# --- (Assuming all indicator functions like add_sma, add_ema, etc., are defined above this line) ---
# --- (For brevity, I'm not re-pasting all of them here, but you should have them in your script) ---

# --- PASTE ALL YOUR INDICATOR CALCULATION FUNCTIONS (add_sma, add_ema, ...) HERE ---
# For example:
def add_sma(df, window=20, column='Close', new_column_name=None):
    if new_column_name is None: new_column_name = f'SMA_{window}'
    df[new_column_name] = df[column].rolling(window=window, min_periods=1).mean()
    return df

def add_ema(df, span=20, column='Close', new_column_name=None):
    if new_column_name is None: new_column_name = f'EMA_{span}'
    df[new_column_name] = df[column].ewm(span=span, adjust=False, min_periods=1).mean()
    return df

def add_rsi(df, window=14, column='Close', new_column_name=None):
    if new_column_name is None: new_column_name = f'RSI_{window}'
    delta = df[column].diff(1)
    gain = delta.where(delta > 0, 0).fillna(0) # fillna for first diff
    loss = -delta.where(delta < 0, 0).fillna(0) # fillna for first diff
    avg_gain = gain.ewm(alpha=1/window, adjust=False, min_periods=1).mean() # Use ewm for standard RSI
    avg_loss = loss.ewm(alpha=1/window, adjust=False, min_periods=1).mean() # Use ewm for standard RSI
    rs = avg_gain / avg_loss
    rs = rs.replace([np.inf, -np.inf], np.nan).fillna(0) # Handle division by zero or NaNs if avg_loss is 0
    rsi = 100 - (100 / (1 + rs))
    # If avg_loss is consistently 0 (all gains), RSI should be 100. If avg_gain is 0 (all losses), RSI is 0.
    rsi = np.where(avg_loss == 0, 100, rsi)
    rsi = np.where(avg_gain == 0, 0, rsi)
    df[new_column_name] = rsi
    return df

def add_bollinger_bands(df, window=20, num_std_dev=2, column='Close', sma_col_name=None, upper_band_col_name=None, lower_band_col_name=None):
    if sma_col_name is None: sma_col_name = f'BB_SMA_{window}'
    if upper_band_col_name is None: upper_band_col_name = f'BB_Upper_{window}'
    if lower_band_col_name is None: lower_band_col_name = f'BB_Lower_{window}'
    df[sma_col_name] = df[column].rolling(window=window, min_periods=1).mean()
    std_dev = df[column].rolling(window=window, min_periods=1).std()
    df[upper_band_col_name] = df[sma_col_name] + (std_dev * num_std_dev)
    df[lower_band_col_name] = df[sma_col_name] - (std_dev * num_std_dev)
    return df

def add_macd(df, short_window=12, long_window=26, signal_window=9, column='Close', macd_line_col_name=None, signal_line_col_name=None, macd_hist_col_name=None):
    if macd_line_col_name is None: macd_line_col_name = f'MACD_{short_window}_{long_window}'
    if signal_line_col_name is None: signal_line_col_name = f'MACD_Signal_{signal_window}'
    if macd_hist_col_name is None: macd_hist_col_name = f'MACD_Hist_{signal_window}'
    ema_short = df[column].ewm(span=short_window, adjust=False, min_periods=1).mean()
    ema_long = df[column].ewm(span=long_window, adjust=False, min_periods=1).mean()
    df[macd_line_col_name] = ema_short - ema_long
    df[signal_line_col_name] = df[macd_line_col_name].ewm(span=signal_window, adjust=False, min_periods=1).mean()
    df[macd_hist_col_name] = df[macd_line_col_name] - df[signal_line_col_name]
    return df

def add_atr(df, window=14, high_col='High', low_col='Low', close_col='Close', atr_col_name=None):
    if atr_col_name is None: atr_col_name = f'ATR_{window}'
    df['_prev_close'] = df[close_col].shift(1)
    df['_h_minus_l'] = df[high_col] - df[low_col]
    df['_h_minus_pc'] = abs(df[high_col] - df['_prev_close'])
    df['_l_minus_pc'] = abs(df[low_col] - df['_prev_close'])
    df['_true_range'] = df[['_h_minus_l', '_h_minus_pc', '_l_minus_pc']].max(axis=1)
    df[atr_col_name] = df['_true_range'].ewm(alpha=1/window, adjust=False, min_periods=1).mean()
    df.drop(columns=['_prev_close', '_h_minus_l', '_h_minus_pc', '_l_minus_pc', '_true_range'], inplace=True)
    return df

def add_stochastic_oscillator(df, k_window=14, d_window=3,
                              high_col='High', low_col='Low', close_col='Close',
                              stoch_k_col_name=None, stoch_d_col_name=None):
    """Adds Stochastic Oscillator (%K and %D) to the DataFrame."""
    if stoch_k_col_name is None: stoch_k_col_name = f'Stoch_%K_{k_window}'
    if stoch_d_col_name is None: stoch_d_col_name = f'Stoch_%D_{d_window}'

    lowest_low = df[low_col].rolling(window=k_window, min_periods=1).min()
    highest_high = df[high_col].rolling(window=k_window, min_periods=1).max()

    # Avoid division by zero if highest_high == lowest_low by adding a small epsilon
    denominator = highest_high - lowest_low
    df[stoch_k_col_name] = ((df[close_col] - lowest_low) / (denominator + 1e-9)) * 100

    # Handle potential NaN/inf values from division and clip to 0-100 range
    df[stoch_k_col_name] = df[stoch_k_col_name].replace([np.inf, -np.inf], np.nan)
    # --- UPDATED LINE TO ADDRESS FutureWarning ---
    df[stoch_k_col_name] = df[stoch_k_col_name].ffill().bfill() # Use .ffill() and .bfill()
    # --- END OF UPDATED LINE ---
    df[stoch_k_col_name] = df[stoch_k_col_name].clip(0, 100) # Ensure values are strictly within 0-100

    df[stoch_d_col_name] = df[stoch_k_col_name].rolling(window=d_window, min_periods=1).mean()
    return df

def add_mfi(df, window=14, high_col='High', low_col='Low', close_col='Close', volume_col='Volume', mfi_col_name=None):
    if mfi_col_name is None: mfi_col_name = f'MFI_{window}'
    df['_typical_price'] = (df[high_col] + df[low_col] + df[close_col]) / 3
    df['_raw_money_flow'] = df['_typical_price'] * df[volume_col]
    price_diff = df['_typical_price'].diff(1)
    df['_positive_money_flow'] = df['_raw_money_flow'].where(price_diff > 0, 0).fillna(0)
    df['_negative_money_flow'] = df['_raw_money_flow'].where(price_diff < 0, 0).fillna(0)
    positive_mf_sum = df['_positive_money_flow'].rolling(window=window, min_periods=1).sum()
    negative_mf_sum = df['_negative_money_flow'].rolling(window=window, min_periods=1).sum()
    money_flow_ratio = positive_mf_sum / (negative_mf_sum + 1e-9) # Add small epsilon to avoid division by zero
    mfi = 100 - (100 / (1 + money_flow_ratio))
    mfi = mfi.replace([np.inf, -np.inf], 100) # If negative_mf_sum was essentially 0
    df[mfi_col_name] = mfi
    df.drop(columns=['_typical_price', '_raw_money_flow', '_positive_money_flow', '_negative_money_flow'], inplace=True)
    return df

def add_roc(df, window=12, column='Close', new_column_name=None):
    if new_column_name is None: new_column_name = f'ROC_{window}'
    df[new_column_name] = df[column].pct_change(periods=window) * 100
    return df

def add_keltner_channels(df, ema_window=20, atr_window=10, atr_multiplier=2, high_col='High', low_col='Low', close_col='Close', middle_col_name=None, upper_col_name=None, lower_col_name=None):
    if middle_col_name is None: middle_col_name = f'KC_Middle_{ema_window}'
    if upper_col_name is None: upper_col_name = f'KC_Upper_{ema_window}'
    if lower_col_name is None: lower_col_name = f'KC_Lower_{ema_window}'
    df[middle_col_name] = df[close_col].ewm(span=ema_window, adjust=False, min_periods=1).mean()
    temp_atr_col = f'_temp_atr_for_kc_{atr_window}'
    df_with_atr = add_atr(df.copy(), window=atr_window, high_col=high_col, low_col=low_col, close_col=close_col, atr_col_name=temp_atr_col) # Use a copy to avoid modifying df inplace during ATR calc
    df[upper_col_name] = df[middle_col_name] + (df_with_atr[temp_atr_col] * atr_multiplier)
    df[lower_col_name] = df[middle_col_name] - (df_with_atr[temp_atr_col] * atr_multiplier)
    return df

def add_volume_sma(df, window=20, volume_col='Volume', new_column_name=None):
    if new_column_name is None: new_column_name = f'Volume_SMA_{window}'
    df[new_column_name] = df[volume_col].rolling(window=window, min_periods=1).mean()
    return df

def add_basic_support_resistance(df, window=20, high_col='High', low_col='Low', support_col_name=None, resistance_col_name=None):
    if support_col_name is None: support_col_name = f'Support_{window}'
    if resistance_col_name is None: resistance_col_name = f'Resistance_{window}'
    df[resistance_col_name] = df[high_col].rolling(window=window, min_periods=1).max().shift(1)
    df[support_col_name] = df[low_col].rolling(window=window, min_periods=1).min().shift(1)
    # print("Note: Basic Support/Resistance added using rolling min/max of prior periods.")
    return df

# END OF INDICATOR FUNCTIONS - PASTE THEM ABOVE THIS LINE

In [7]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta # If you're using these elsewhere
# from google.colab import drive # If you're using drive

In [9]:
import pandas as pd # Ensure pandas is imported
import os # Ensure os is imported
import numpy as np # For np.nan

# This script assumes:
# 1. You have run the Google Drive mount cell.
# 2. Your indicator functions (add_sma, add_rsi, etc.) are defined above this block.
# 3. Your master_indicator_configs dictionary is defined above this block.

# --- Configuration for File Paths on Google Drive ---
# Define the pairs, timeframes, and duration from your CCXT script's output
TARGET_PAIRS = ["BTC_USDT", "ETH_USDT", "SOL_USDT"] # e.g., BTC_USDT, ETH_USDT
TIMEFRAMES_TO_PROCESS = ["1d", "1h", "15m"] # CCXT timeframe codes, e.g., "1d", "1h", "15m"
DURATION_TAG = "4years" # From your CCXT script's naming

GDRIVE_BASE_DATA_PATH = "/content/drive/MyDrive/CryptoDataCCXT/" # Source of CCXT data
GDRIVE_OUTPUT_PATH = "/content/drive/MyDrive/CryptoDataCCXT/" # Output for this script

if not os.path.exists(GDRIVE_OUTPUT_PATH):
    os.makedirs(GDRIVE_OUTPUT_PATH)
    print(f"Created output directory in Google Drive: {GDRIVE_OUTPUT_PATH}")
else:
    print(f"Output directory in Google Drive already exists or is accessible: {GDRIVE_OUTPUT_PATH}")

for pair_symbol in TARGET_PAIRS:
    # Extract base coin symbol for looking up in master_indicator_configs (e.g., "BTC" from "BTC_USDT")
    base_coin_symbol = pair_symbol.split('_')[0] # Assumes format like COIN_QUOTE (e.g., BTC_USDT)
                                               # Adjust if your pair_symbol format is different (e.g. BTCUSD)

    for timeframe_ccxt in TIMEFRAMES_TO_PROCESS:
        print(f"\n\nProcessing data for: {pair_symbol} ({timeframe_ccxt})")
        print("==================================================")

        data_file_name = f"{pair_symbol}_{timeframe_ccxt}_{DURATION_TAG}.csv"
        full_data_path = os.path.join(GDRIVE_BASE_DATA_PATH, data_file_name)

        try:
            # The CCXT script saved 'datetime_utc' as the human-readable timestamp column
            df_original = pd.read_csv(full_data_path, index_col='datetime_utc', parse_dates=True)
            print(f"Successfully loaded data for {pair_symbol} ({timeframe_ccxt}) from: {full_data_path}")

            print(f"DEBUG: Original columns found: {df_original.columns.tolist()}")

            df = df_original.copy()

            # --- FIX 1: Rename columns to capitalized versions for indicator functions ---
            # CCXT output columns are: 'open', 'high', 'low', 'close', 'volume', 'timestamp' (ms)
            rename_map = {
                'open': 'Open',
                'high': 'High',
                'low': 'Low',
                'close': 'Close',
                'volume': 'Volume' # CCXT data should have 'volume'
            }
            # Only rename columns that actually exist in the DataFrame with lowercase names
            columns_to_rename_now = {k: v for k, v in rename_map.items() if k in df.columns}
            if columns_to_rename_now:
                df.rename(columns=columns_to_rename_now, inplace=True)
                print(f"DEBUG: Columns after renaming: {df.columns.tolist()}")

        except FileNotFoundError:
            print(f"ERROR: Data file not found for {pair_symbol} ({timeframe_ccxt}) at '{full_data_path}'. Skipping.")
            continue
        except Exception as e:
            print(f"ERROR: Could not load data for {pair_symbol} ({timeframe_ccxt}) from '{full_data_path}': {e}. Skipping.")
            continue

        # --- FIX 2: Define required_cols (after renaming) ---
        # For CCXT data, 'Volume' should be present after renaming.
        required_cols_for_this_file = ['Open', 'High', 'Low', 'Close', 'Volume']

        missing_required_cols = [col for col in required_cols_for_this_file if col not in df.columns]
        if missing_required_cols:
            print(f"ERROR: DataFrame for {pair_symbol} ({timeframe_ccxt}) is missing one or more required OHLCV columns: {missing_required_cols}. Actual columns: {df.columns.tolist()}. Skipping.")
            continue

        # --- Get Coin-Specific Indicator Configuration ---
        # Using base_coin_symbol (e.g., "BTC") for lookup
        if base_coin_symbol in master_indicator_configs:
            current_config = master_indicator_configs[base_coin_symbol]
            print(f"Using specific indicator configurations for {base_coin_symbol} (from {pair_symbol}).")
        else:
            print(f"WARNING: No specific indicator configuration found for {base_coin_symbol} (from {pair_symbol}) in 'master_indicator_configs'. Skipping indicators for this file.")
            # Decide if you want to save the file without indicators or skip saving
            # For now, let's skip applying indicators and thus not save a new file
            continue

        print(f"\nApplying technical indicators for {pair_symbol} ({timeframe_ccxt})...")

        # --- Apply Indicators (with checks for 'Volume' where needed) ---
        # Assumes indicator functions like add_sma, add_rsi, add_mfi are defined elsewhere
        # and expect uppercase column names ('Open', 'Close', 'Volume', etc.)

        # --- Moving Averages (don't need Volume) ---
        if "sma_short" in current_config:
            params = current_config["sma_short"]
            df = add_sma(df, **params, new_column_name=f'{base_coin_symbol}_SMA_{params.get("window", "short")}')
        if "sma_long" in current_config:
            params = current_config["sma_long"]
            df = add_sma(df, **params, new_column_name=f'{base_coin_symbol}_SMA_{params.get("window", "long")}')
        if "ema_short" in current_config:
            params = current_config["ema_short"]
            df = add_ema(df, **params, new_column_name=f'{base_coin_symbol}_EMA_{params.get("span", "short")}')
        if "ema_long" in current_config:
            params = current_config["ema_long"]
            df = add_ema(df, **params, new_column_name=f'{base_coin_symbol}_EMA_{params.get("span", "long")}')

        # --- Other Non-Volume Dependent Indicators ---
        if "rsi" in current_config: df = add_rsi(df, **current_config["rsi"])
        if "bollinger" in current_config: df = add_bollinger_bands(df, **current_config["bollinger"])
        if "macd" in current_config: df = add_macd(df, **current_config["macd"])
        if "atr" in current_config: df = add_atr(df, **current_config["atr"])
        if "stochastic" in current_config: df = add_stochastic_oscillator(df, **current_config["stochastic"])
        if "roc" in current_config: df = add_roc(df, **current_config["roc"])
        if "keltner" in current_config: df = add_keltner_channels(df, **current_config["keltner"])
        if "support_resistance" in current_config: df = add_basic_support_resistance(df, **current_config["support_resistance"])

        # --- Indicators REQUIRING Volume (MFI, Volume SMA) ---
        if 'Volume' in df.columns: # This check is robust
            print(f"DEBUG: 'Volume' column available for {pair_symbol}. Proceeding with volume-based indicators.")
            if "mfi" in current_config:
                df = add_mfi(df, **current_config["mfi"])
            if "volume_sma" in current_config:
                df = add_volume_sma(df, **current_config["volume_sma"])
        else:
            # This block might be less likely to hit if using CCXT data which includes volume
            print(f"WARNING: 'Volume' column NOT found for {pair_symbol} even after renaming. Skipping MFI and Volume SMA.")
            if "mfi" in current_config:
                mfi_window = current_config["mfi"].get("window", 14)
                df[f'MFI_{mfi_window}'] = np.nan
            if "volume_sma" in current_config:
                vol_sma_window = current_config["volume_sma"].get("window", 20)
                df[f'Volume_SMA_{vol_sma_window}'] = np.nan

        # --- Output and Save Results ---
        print(f"\n--- {pair_symbol} ({timeframe_ccxt}): DataFrame with Technical Indicators (First 3 rows) ---")
        pd.set_option('display.max_columns', None) # Show all columns for head()
        print(df.head(3))
        pd.reset_option('display.max_columns')

        print(f"\n--- {pair_symbol} ({timeframe_ccxt}): DataFrame Info ---")
        df.info()

        # Make output filename more descriptive
        output_file_name = f"{pair_symbol}_{timeframe_ccxt}_with_indicators.csv"
        full_output_path = os.path.join(GDRIVE_OUTPUT_PATH, output_file_name)
        try:
            df.to_csv(full_output_path)
            print(f"Successfully saved DataFrame for {pair_symbol} ({timeframe_ccxt}) with indicators to: {full_output_path}")
        except Exception as e:
            print(f"ERROR: Could not save DataFrame for {pair_symbol} ({timeframe_ccxt}) to '{full_output_path}': {e}")

print("\n\n--- All Coin Processing Complete ---")

Output directory in Google Drive already exists or is accessible: /content/drive/MyDrive/CryptoDataCCXT/


Processing data for: BTC_USDT (1d)
Successfully loaded data for BTC_USDT (1d) from: /content/drive/MyDrive/CryptoDataCCXT/BTC_USDT_1d_4years.csv
DEBUG: Original columns found: ['open', 'high', 'low', 'close', 'volume', 'timestamp']
DEBUG: Columns after renaming: ['Open', 'High', 'Low', 'Close', 'Volume', 'timestamp']
Using specific indicator configurations for BTC (from BTC_USDT).

Applying technical indicators for BTC_USDT (1d)...


NameError: name 'add_sma' is not defined

In [10]:
import pandas as pd
import os
from google.colab import drive # For Google Drive access


# --- Configuration for Verification ---
# This should be the EXACT path where your INDICATOR SCRIPT (processing individual timeframes) saves its output.
GDRIVE_INDICATORS_OUTPUT_PATH = "/content/drive/MyDrive/CryptoDataCCXT/" # As per your log

# Define the files your indicator script is expected to generate
TARGET_PAIRS_FOR_VERIFICATION = ["BTC_USDT", "ETH_USDT", "SOL_USDT"]
TIMEFRAMES_FOR_VERIFICATION = ["1d", "1h", "15m"] # CCXT timeframe codes
# The suffix added by your indicator script (the one we worked on before this verification script)
OUTPUT_FILENAME_SUFFIX = "with_indicators.csv"

EXPECTED_FILES_TO_VERIFY = []
for pair in TARGET_PAIRS_FOR_VERIFICATION:
    for tf in TIMEFRAMES_FOR_VERIFICATION:
        # Filename format from your indicator processing script that adds indicators
        filename = f"{pair}_{tf}_{OUTPUT_FILENAME_SUFFIX}"
        EXPECTED_FILES_TO_VERIFY.append(filename)

print("Expected files to verify (individual timeframe files with indicators):")
for f_name in EXPECTED_FILES_TO_VERIFY:
    print(f"- {f_name}")

# --- Main Verification Logic ---
for file_to_verify in EXPECTED_FILES_TO_VERIFY:
    full_file_path = os.path.join(GDRIVE_INDICATORS_OUTPUT_PATH, file_to_verify)
    # Extract base coin symbol and timeframe for context in checks
    parts = file_to_verify.split('_')
    base_coin_symbol = parts[0] # e.g., BTC
    timeframe_shortcode = parts[2] # e.g., 1d, 1h, 15m

    print(f"\n\n======================================================================")
    print(f"--- Verifying File: {full_file_path} ---")
    print(f"======================================================================")

    if not os.path.exists(full_file_path):
        print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(f"ERROR: File not found at {full_file_path}")
        print(f"Please ensure:")
        print(f"1. The indicator processing script has run successfully and created this file.")
        print(f"2. The path above is correct and your Google Drive is mounted.")
        print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        continue # Skip to the next file
    else:
      try:
          # Load the CSV. Index is 'datetime_utc'.
          df = pd.read_csv(full_file_path, index_col='datetime_utc', parse_dates=True)
          print("Successfully loaded the file.")

          if df.empty:
              print("WARNING: The file is empty.")
          else:
              print(f"\n1. Shape of the DataFrame (rows, columns): {df.shape}")
              if df.shape[0] == 0:
                  print("WARNING: DataFrame has 0 rows.")
              if df.shape[1] == 0:
                  print("WARNING: DataFrame has 0 columns.")

              print("\n2. First 3 rows (transposed if many columns for readability):")
              try:
                  pd.set_option('display.max_columns', None)
                  if df.shape[1] > 15 : # If more than 15 columns, transpose
                      print(df.head(3).T)
                  else:
                      print(df.head(3))
              finally:
                  pd.reset_option('display.max_columns')

              print("\n3. Index details:")
              print(f"   Index name: {df.index.name}")
              print(f"   Index data type: {df.index.dtype}") # Should be datetime64[ns, UTC]
              if not df.empty:
                  print(f"   Is index monotonic increasing? {df.index.is_monotonic_increasing}")
                  if not df.index.is_monotonic_increasing and len(df) > 1:
                      print(f"   WARNING: Index is not strictly monotonic increasing!")
                  print(f"   Date range: {df.index.min()} to {df.index.max()}")
              print(f"   Number of rows: {len(df)}")
              try: # Check frequency
                  freq = pd.infer_freq(df.index)
                  print(f"   Inferred frequency of index: {freq}")
                  expected_freq_map = {'1d': 'D', '1h': 'H', '15m': '15T'} # 'T' or 'min' for minutes
                  expected_tf_freq = expected_freq_map.get(timeframe_shortcode)
                  if freq != expected_tf_freq and len(df) > 1:
                      # Handle variations like 'BH' for business hour, '15min' vs '15T'
                      is_freq_ok = False
                      if timeframe_shortcode == '1h' and freq in ['H', 'BH']: is_freq_ok = True
                      elif timeframe_shortcode == '15m' and freq in ['15T', '15min']: is_freq_ok = True
                      elif timeframe_shortcode == '1d' and freq == 'D': is_freq_ok = True

                      if not is_freq_ok:
                          print(f"   WARNING: Index frequency ({freq}) does not match expected for timeframe {timeframe_shortcode} (e.g., {expected_tf_freq}).")
              except Exception as e:
                  print(f"   Could not infer frequency: {e}")


              print("\n4. Column Name Checks (No Prefixes Expected):")
              all_columns = df.columns.tolist()
              print(f"   Total columns found: {len(all_columns)}")
              # Check for essential OHLCV columns (should be uppercase after your processing script)
              essential_ohlcv = ['Open', 'High', 'Low', 'Close', 'Volume', 'timestamp']
              missing_essentials = []
              for col_name in essential_ohlcv:
                  if col_name not in df.columns:
                      missing_essentials.append(col_name)
              if missing_essentials:
                  print(f"     WARNING: Missing essential OHLCV or original timestamp columns: {missing_essentials}")
              else:
                  print(f"     All essential OHLCV + original timestamp columns found.")

              # Check for example indicator columns (these names depend on your master_indicator_configs)
              # This is a simple check; a full check would parse master_indicator_configs
              example_indicator_suffixes = ['_SMA_', '_EMA_', 'RSI_', 'BB_SMA_', 'MACD_', 'ATR_', 'Stoch_%K', 'MFI_', 'ROC_']
              found_indicator_examples = []
              for col in all_columns:
                  if any(suffix in col for suffix in example_indicator_suffixes):
                      found_indicator_examples.append(col)

              if found_indicator_examples:
                  print(f"   Found example indicator columns (first few): {found_indicator_examples[:5]}")
                  if len(found_indicator_examples) == 0 : # Corrected from > 0
                     print(f"   WARNING: No example indicator columns found! Check indicator generation.")
              else:
                  print(f"   WARNING: No example indicator columns (based on suffixes) found! All columns: {all_columns}")


              print("\n5. NaN Value Analysis (Summary):")
              total_cells = df.shape[0] * df.shape[1]
              total_nan_cells = df.isnull().sum().sum()
              if total_cells > 0 :
                  percentage_nan = (total_nan_cells / total_cells) * 100
                  print(f"   Total NaN values: {total_nan_cells} (out of {total_cells} cells)")
                  print(f"   Percentage of NaN values: {percentage_nan:.2f}%")
              else:
                  print(f"   DataFrame is empty, no NaN values to analyze.")

              nan_summary_per_col = df.isnull().sum()
              cols_with_nans = nan_summary_per_col[nan_summary_per_col > 0].sort_values(ascending=False)
              if not cols_with_nans.empty:
                  print("   Top columns with NaN values and their counts (max 10 shown):")
                  print(cols_with_nans.head(10))
                  if len(cols_with_nans) > 10:
                      print(f"   ... and {len(cols_with_nans) - 10} more columns with NaNs.")
                  print("   (Note: NaNs are expected at the start of indicator series due to lookback periods.)")
              elif not df.empty :
                  print("   No NaN values found in any column.")

      except pd.errors.EmptyDataError:
          print(f"ERROR: The file '{full_file_path}' is empty and cannot be parsed by Pandas.")
      except Exception as e:
          print(f"ERROR: An error occurred while reading or processing '{full_file_path}': {e}")

print("\n\n--- Verification Script for Individual Timeframe Files Complete ---")

Expected files to verify (individual timeframe files with indicators):
- BTC_USDT_1d_with_indicators.csv
- BTC_USDT_1h_with_indicators.csv
- BTC_USDT_15m_with_indicators.csv
- ETH_USDT_1d_with_indicators.csv
- ETH_USDT_1h_with_indicators.csv
- ETH_USDT_15m_with_indicators.csv
- SOL_USDT_1d_with_indicators.csv
- SOL_USDT_1h_with_indicators.csv
- SOL_USDT_15m_with_indicators.csv


--- Verifying File: /content/drive/MyDrive/CryptoDataCCXT/BTC_USDT_1d_with_indicators.csv ---
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ERROR: File not found at /content/drive/MyDrive/CryptoDataCCXT/BTC_USDT_1d_with_indicators.csv
Please ensure:
1. The indicator processing script has run successfully and created this file.
2. The path above is correct and your Google Drive is mounted.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


--- Verifying File: /content/drive/MyDrive/CryptoDataCCXT/BTC_USDT_1h_with_indicators.csv ---
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [11]:
import pandas as pd
import os

def read_and_display_single_csv(file_path, num_rows_to_display=25):
    """
    Reads a single CSV file and displays basic information about it.

    Args:
        file_path (str): The full path to the CSV file.
        num_rows_to_display (int): The number of rows to display from the head.
    """
    print(f"--- Attempting to read file: {file_path} ---")

    if not os.path.exists(file_path):
        print(f"ERROR: File not found at '{file_path}'.")
        print("Please ensure the file path is correct and the file exists.")
        return

    if not os.path.isfile(file_path):
        print(f"ERROR: '{file_path}' is not a file.")
        return

    try:
        # Attempt to read the CSV file.
        # You might need to specify other parameters for pd.read_csv() depending on your file's format,
        # such as:
        # - sep=',' (or other delimiter like '\t', ';')
        # - header=0 (row number to use as column names)
        # - index_col='your_index_column_name' (if you have a specific index column)
        # - parse_dates=['your_date_column'] (to parse specific columns as dates)
        df = pd.read_csv(file_path)

        if df.empty:
            print("WARNING: The CSV file is empty (contains no data or only headers).")
        else:
            print("\nSuccessfully read the file.")

            print(f"\n1. First {num_rows_to_display} rows of the DataFrame:")
            print(df.head(num_rows_to_display))

            print(f"\n2. Shape of the DataFrame (rows, columns): {df.shape}")

            print("\n3. DataFrame Info (column data types and non-null values):")
            df.info()

            print("\n4. Basic descriptive statistics for numerical columns:")
            print(df.describe())

    except pd.errors.EmptyDataError:
        print("ERROR: The file is empty and could not be parsed by Pandas (no columns found).")
    except Exception as e:
        print(f"An error occurred while reading or processing the file: {e}")

# --- Main Execution Example ---
if __name__ == "__main__":
    # **IMPORTANT**: Replace this with the actual path to YOUR file!
    # Example for a file in the current directory:
    # file_to_read = "my_data.csv"

    # Example for a file in a Colab session (after uploading or generation):
    # file_to_read = "/content/BTC_USDT_1d_with_indicators.csv"

    # Example for a file on Google Drive (ensure Drive is mounted in Colab first):
    # Make sure to run the drive mount cell in Colab:
    # from google.colab import drive
    # drive.mount('/content/drive')
    # file_to_read = "/content/drive/MyDrive/CryptoDataCCXT/with_indicators/BTC_USDT_1d_with_indicators.csv"

    # --- SET YOUR FILE PATH HERE ---
    # Please update this path to point to the specific file you want to read.
    # For instance, if you just ran the indicator script and want to check one of its outputs:
    file_to_read = "/content/drive/MyDrive/CryptoDataCCXT/with_indicators/BTC_USDT_1d_with_indicators.csv"
    # Or, if you are not in Colab and the file is in the same directory as the script:
    # file_to_read = "BTC_USDT_1d_with_indicators.csv"


    # Check if running in Colab and the path looks like a Drive path, then try to mount.
    # This is a basic check; for more robust Colab/Drive handling, you might expand this.
    if file_to_read.startswith("/content/drive/"):
        try:
            from google.colab import drive
            drive.mount('/content/drive', force_remount=True) # Using force_remount for convenience
            print("Google Drive mounted (or re-mounted).")
        except ModuleNotFoundError:
            print("Warning: Not in a Colab environment, but path looks like a Colab Drive path. Proceeding...")
        except Exception as e:
            print(f"Warning: Could not mount Google Drive: {e}. Ensure it's mounted if the path requires it.")


    read_and_display_single_csv(file_to_read)

    print("\n--- Script execution finished ---")


--- Attempting to read file: /content/drive/MyDrive/CryptoDataCCXT/with_indicators/BTC_USDT_1d_with_indicators.csv ---
ERROR: File not found at '/content/drive/MyDrive/CryptoDataCCXT/with_indicators/BTC_USDT_1d_with_indicators.csv'.
Please ensure the file path is correct and the file exists.

--- Script execution finished ---


In [14]:
import pandas as pd
import os
import numpy as np # <<< Ensure numpy is imported for np.nan
from google.colab import drive # For Google Drive access

# --- Step 1: Mount Google Drive (Assumed to be run in a previous cell or handled)---
try:
    drive.mount('/content/drive', force_remount=False)
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    raise RuntimeError("Google Drive could not be mounted. Script aborted.")

# --- Step 2: Ensure Indicator Functions and Master Config are Defined ---
# IMPORTANT: Your functions (add_sma, add_ema, ..., add_basic_support_resistance)
# AND your 'master_indicator_configs' dictionary MUST be defined in your
# Colab notebook BEFORE this script cell is executed.

# --- Configuration for the specific file to process ---
BASE_OHLCV_PATH = "/content/drive/MyDrive/CryptoDataCCXT/" # Path to original CCXT data (without indicators)
FILE_TO_PROCESS_PAIR = "BTC_USDT"
FILE_TO_PROCESS_TIMEFRAME = "1d" # Example: "1d", "1h", or "15m"
FILE_TO_PROCESS_DURATION_TAG = "4years"
FILE_TO_PROCESS = os.path.join(BASE_OHLCV_PATH, f"{FILE_TO_PROCESS_PAIR}_{FILE_TO_PROCESS_TIMEFRAME}_{FILE_TO_PROCESS_DURATION_TAG}.csv")
BASE_COIN_SYMBOL = FILE_TO_PROCESS_PAIR.split('_')[0] # e.g., "BTC" from "BTC_USDT"

print(f"\n--- Processing File to Generate/Inspect Specific Indicator Data ---")
print(f"File: {FILE_TO_PROCESS}")
print(f"Base Coin for Config: {BASE_COIN_SYMBOL}")

if not os.path.exists(FILE_TO_PROCESS):
    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print(f"ERROR: File not found at {FILE_TO_PROCESS}")
    print(f"Please ensure the path and filename are correct, it's a BASE OHLCV file,")
    print(f"and your Google Drive is mounted.")
    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
else:
    try:
        # Load the base OHLCV CSV file.
        df = pd.read_csv(FILE_TO_PROCESS)

        # Standardize Datetime Index
        if 'datetime_utc' in df.columns:
            df['datetime_utc'] = pd.to_datetime(df['datetime_utc'])
            df.set_index('datetime_utc', inplace=True)
            print("Used 'datetime_utc' column as DatetimeIndex.")
        elif 'timestamp' in df.columns and not isinstance(df.index, pd.DatetimeIndex):
            df['datetime_utc'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
            df.set_index('datetime_utc', inplace=True)
            print("Used 'timestamp' (ms) column to create and set DatetimeIndex 'datetime_utc'.")
        else:
            print("Warning: No 'datetime_utc' or 'timestamp' column found to set as DatetimeIndex. Ensure your CSV has one for time series analysis.")


        print(f"Successfully loaded: {FILE_TO_PROCESS}")

        # Ensure OHLCV columns are uppercase for indicator functions
        rename_map = {
            'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'
        }
        cols_to_rename = {k: v for k, v in rename_map.items() if k in df.columns}
        if cols_to_rename:
            df.rename(columns=cols_to_rename, inplace=True)
            print(f"Renamed base OHLCV columns to uppercase: {list(cols_to_rename.values())}")

        print(f"Columns after potential rename and index setting: {df.columns.tolist()}")

        # --- Apply/Re-apply All Indicators from Master Config ---
        if BASE_COIN_SYMBOL in master_indicator_configs: # master_indicator_configs must be defined
            current_config = master_indicator_configs[BASE_COIN_SYMBOL]

            print(f"\nApplying all configured indicators for {BASE_COIN_SYMBOL}...")
            # Work on a copy if you want to preserve the original df in this cell's scope
            # or if your indicator functions modify inplace in unexpected ways.
            # Since your functions return df, direct assignment is also fine.
            df_with_indicators = df.copy()

            # --- Moving Averages ---
            if "sma_short" in current_config:
                params = current_config["sma_short"]
                col_name = f'{BASE_COIN_SYMBOL}_SMA_{params.get("window")}' # .get("window", default_val) if window might be missing
                if col_name in df_with_indicators.columns: print(f"Info: Column {col_name} exists. Re-calculating.")
                df_with_indicators = add_sma(df_with_indicators, **params, new_column_name=col_name) # Assumes add_sma and others are defined
            if "sma_long" in current_config:
                params = current_config["sma_long"]
                col_name = f'{BASE_COIN_SYMBOL}_SMA_{params.get("window")}'
                if col_name in df_with_indicators.columns: print(f"Info: Column {col_name} exists. Re-calculating.")
                df_with_indicators = add_sma(df_with_indicators, **params, new_column_name=col_name)
            if "ema_short" in current_config:
                params = current_config["ema_short"]
                col_name = f'{BASE_COIN_SYMBOL}_EMA_{params.get("span")}'
                if col_name in df_with_indicators.columns: print(f"Info: Column {col_name} exists. Re-calculating.")
                df_with_indicators = add_ema(df_with_indicators, **params, new_column_name=col_name)
            if "ema_long" in current_config:
                params = current_config["ema_long"]
                col_name = f'{BASE_COIN_SYMBOL}_EMA_{params.get("span")}'
                if col_name in df_with_indicators.columns: print(f"Info: Column {col_name} exists. Re-calculating.")
                df_with_indicators = add_ema(df_with_indicators, **params, new_column_name=col_name)

            # --- Other Non-Volume Dependent Indicators ---
            if "rsi" in current_config:
                params = current_config["rsi"]
                col_name = f'RSI_{params.get("window", 14)}' # Defaulting for print, add_rsi handles its own name
                if col_name in df_with_indicators.columns: print(f"Info: Column {col_name} (or similar from add_rsi) exists. Re-calculating.")
                df_with_indicators = add_rsi(df_with_indicators, **params) # add_rsi creates its own default name if new_column_name not passed

            if "bollinger" in current_config:
                params = current_config["bollinger"]
                # add_bollinger_bands creates multiple columns (BB_SMA_*, BB_Upper_*, BB_Lower_*)
                # We'll just check one for the print message
                col_name_check = f'BB_SMA_{params.get("window", 20)}'
                if col_name_check in df_with_indicators.columns: print(f"Info: Bollinger Band columns (e.g. {col_name_check}) may exist. Re-calculating.")
                df_with_indicators = add_bollinger_bands(df_with_indicators, **params)

            if "macd" in current_config:
                params = current_config["macd"]
                col_name_check = f'MACD_{params.get("short_window",12)}_{params.get("long_window",26)}'
                if col_name_check in df_with_indicators.columns: print(f"Info: MACD columns (e.g. {col_name_check}) may exist. Re-calculating.")
                df_with_indicators = add_macd(df_with_indicators, **params)

            if "atr" in current_config:
                params = current_config["atr"]
                col_name = f'ATR_{params.get("window", 14)}'
                if col_name in df_with_indicators.columns: print(f"Info: Column {col_name} exists. Re-calculating.")
                df_with_indicators = add_atr(df_with_indicators, **params)

            if "stochastic" in current_config:
                params = current_config["stochastic"]
                col_name_check = f'Stoch_%K_{params.get("k_window",14)}'
                if col_name_check in df_with_indicators.columns: print(f"Info: Stochastic columns (e.g. {col_name_check}) may exist. Re-calculating.")
                df_with_indicators = add_stochastic_oscillator(df_with_indicators, **params)

            if "roc" in current_config:
                params = current_config["roc"]
                col_name = f'ROC_{params.get("window")}' # Assumes "window" is always in params for ROC
                if col_name in df_with_indicators.columns: print(f"Info: Column {col_name} exists. Re-calculating.")
                df_with_indicators = add_roc(df_with_indicators, **params)

            if "keltner" in current_config:
                params = current_config["keltner"]
                col_name_check = f'KC_Middle_{params.get("ema_window",20)}'
                if col_name_check in df_with_indicators.columns: print(f"Info: Keltner Channel columns (e.g. {col_name_check}) may exist. Re-calculating.")
                df_with_indicators = add_keltner_channels(df_with_indicators, **params)

            if "support_resistance" in current_config:
                params = current_config["support_resistance"]
                col_name_check = f'Support_{params.get("window",20)}'
                if col_name_check in df_with_indicators.columns: print(f"Info: Support/Resistance columns (e.g. {col_name_check}) may exist. Re-calculating.")
                df_with_indicators = add_basic_support_resistance(df_with_indicators, **params)

            # --- Indicators REQUIRING Volume (MFI, Volume SMA) ---
            if 'Volume' in df_with_indicators.columns:
                print(f"DEBUG: 'Volume' column available. Proceeding with volume-based indicators.")
                if "mfi" in current_config:
                    params = current_config["mfi"]
                    col_name = f'MFI_{params.get("window", 14)}'
                    if col_name in df_with_indicators.columns: print(f"Info: Column {col_name} exists. Re-calculating.")
                    df_with_indicators = add_mfi(df_with_indicators, **params)

                if "volume_sma" in current_config:
                    params = current_config["volume_sma"]
                    col_name = f'Volume_SMA_{params.get("window", 20)}'
                    if col_name in df_with_indicators.columns: print(f"Info: Column {col_name} exists. Re-calculating.")
                    df_with_indicators = add_volume_sma(df_with_indicators, **params)
            else:
                print(f"WARNING: 'Volume' column NOT found in DataFrame. Skipping MFI and Volume SMA.")
                # Add NaN columns if they were expected but volume was missing
                if "mfi" in current_config:
                    params = current_config["mfi"]
                    df_with_indicators[f'MFI_{params.get("window", 14)}'] = np.nan
                if "volume_sma" in current_config:
                    params = current_config["volume_sma"]
                    df_with_indicators[f'Volume_SMA_{params.get("window", 20)}'] = np.nan

            print("All configured indicators applied/re-applied.")

            # --- Display and Inspect the Indicator Data ---
            print(f"\n--- Specific Indicator Data for {BASE_COIN_SYMBOL} (from {FILE_TO_PROCESS_PAIR} {FILE_TO_PROCESS_TIMEFRAME}) ---")
            print(f"\n1. DataFrame Shape (rows, columns): {df_with_indicators.shape}")

            print("\n2. First 5 rows (transposed for readability if many columns):")
            pd.set_option('display.max_columns', None)
            if df_with_indicators.shape[1] > 10: print(df_with_indicators.head().T)
            else: print(df_with_indicators.head())
            pd.reset_option('display.max_columns')

            print("\n3. Last 5 rows (transposed for readability if many columns):")
            pd.set_option('display.max_columns', None)
            if df_with_indicators.shape[1] > 10: print(df_with_indicators.tail().T)
            else: print(df_with_indicators.tail())
            pd.reset_option('display.max_columns')

            print("\n4. All Column Names after adding indicators:")
            print(df_with_indicators.columns.tolist())

            print("\n5. Info for DataFrame with indicators:")
            df_with_indicators.info()

            # Optionally, save this newly processed DataFrame
            output_filename_processed = os.path.join(BASE_OHLCV_PATH, "with_indicators_calculated_now", f"{FILE_TO_PROCESS_PAIR}_{FILE_TO_PROCESS_TIMEFRAME}_calculated_indicators.csv")
            os.makedirs(os.path.dirname(output_filename_processed), exist_ok=True)
            df_with_indicators.to_csv(output_filename_processed)
            print(f"\nSaved DataFrame with calculated indicators to: {output_filename_processed}")

        else:
            print(f"ERROR: No configuration found for '{BASE_COIN_SYMBOL}' in master_indicator_configs.")

    except FileNotFoundError:
        # This is already handled by the outer check, but good for robustness
        print(f"ERROR: File not found during processing: {FILE_TO_PROCESS}")
    except Exception as e:
        print(f"An error occurred during data processing or indicator application: {e}")
        import traceback
        traceback.print_exc() # Print full traceback for debugging

print("\n--- Script to generate/inspect specific indicator data complete ---")

Error mounting Google Drive: Mountpoint must not already contain files


RuntimeError: Google Drive could not be mounted. Script aborted.

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import drive

# --- Step 1: Mount Google Drive ---
try:
    drive.mount('/content/drive', force_remount=False)
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    raise RuntimeError("Google Drive could not be mounted. Script aborted.")

# --- Step 2: Load Your Data with Indicators ---
# This file should already contain your calculated SMAs
GDRIVE_INDICATORS_PATH = "/content/drive/MyDrive/CryptoDataCCXT/with_indicators/"
file_to_process = "BTC_USDT_1d_with_indicators.csv" # Using the daily BTC file
full_file_path = os.path.join(GDRIVE_INDICATORS_PATH, file_to_process)

print(f"\n--- Loading data for strategy: {full_file_path} ---")

if not os.path.exists(full_file_path):
    print(f"ERROR: File not found at {full_file_path}. Cannot proceed.")
else:
    try:
        df = pd.read_csv(full_file_path, index_col='datetime_utc', parse_dates=True)
        print("Successfully loaded the data.")
        print(f"Data shape: {df.shape}")
        print("First 5 rows:")
        print(df.head())

        # --- Step 3: Define Indicator Columns for the Strategy ---
        # These names should match the columns in your CSV file.
        # These are based on your master_indicator_configs for BTC:
        # "BTC": { "sma_short": {"window": 10, "column": "Close"},
        #          "sma_long":  {"window": 50, "column": "Close"}, ... }
        short_sma_col = 'BTC_SMA_10' # Adjust if your sma_short for BTC has a different window
        long_sma_col = 'BTC_SMA_50'  # Adjust if your sma_long for BTC has a different window

        # Check if required SMA columns exist
        if not all(col in df.columns for col in [short_sma_col, long_sma_col]):
            print(f"ERROR: Required SMA columns ('{short_sma_col}', '{long_sma_col}') not found in the DataFrame.")
            print(f"Available columns: {df.columns.tolist()}")
            # Exit or handle error appropriately
        else:
            print(f"\nUsing short SMA: {short_sma_col}, Long SMA: {long_sma_col}")

            # --- Step 4: Generate Trading Signals (SMA Crossover) ---
            # Initialize signal column: 0 = Hold, 1 = Buy, -1 = Sell (to exit long)
            df['signal'] = 0

            # Generate buy signals
            # Condition 1: Short SMA crosses above Long SMA
            # (Short SMA > Long SMA in current period) AND (Short SMA < Long SMA in previous period)
            df['buy_condition'] = (df[short_sma_col] > df[long_sma_col]) & \
                                  (df[short_sma_col].shift(1) < df[long_sma_col].shift(1))
            df.loc[df['buy_condition'], 'signal'] = 1

            # Generate sell signals (to exit a long position)
            # Condition 1: Short SMA crosses below Long SMA
            # (Short SMA < Long SMA in current period) AND (Short SMA > Long SMA in previous period)
            df['sell_condition'] = (df[short_sma_col] < df[long_sma_col]) & \
                                   (df[short_sma_col].shift(1) > df[long_sma_col].shift(1))
            df.loc[df['sell_condition'], 'signal'] = -1

            # Remove temporary condition columns
            df.drop(columns=['buy_condition', 'sell_condition'], inplace=True)

            # --- Step 5: Display Results ---
            print("\n--- Strategy Signals Generated (SMA Crossover) ---")

            # Show rows where signals are generated
            signals_df = df[df['signal'] != 0]
            print(f"Number of buy signals: {len(df[df['signal'] == 1])}")
            print(f"Number of sell signals (exit long): {len(df[df['signal'] == -1])}")

            print("\nFirst 10 signals generated:")
            print(signals_df[['Close', short_sma_col, long_sma_col, 'signal']].head(10))

            print("\nLast 10 signals generated:")
            print(signals_df[['Close', short_sma_col, long_sma_col, 'signal']].tail(10))

            # You can save this DataFrame with signals for backtesting
            output_filename = os.path.join(GDRIVE_INDICATORS_PATH, "BTC_USDT_1d_sma_crossover_signals.csv")
            df.to_csv(output_filename)
            print(f"\nSaved DataFrame with signals to: {output_filename}")

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()

print("\n--- Strategy signal generation script complete ---")

In [ ]:
!pip install backtesting

In [ ]:
import os
from google.colab import drive

def list_all_files_in_drive(start_path):
    """
    Lists all files recursively starting from the given path in Google Drive.

    Args:
        start_path (str): The starting directory path within your mounted Google Drive
                          (e.g., "/content/drive/MyDrive/" or a subfolder like
                          "/content/drive/MyDrive/MyProjectFolder").
    """
    print(f"\n--- Listing all files starting from: {start_path} ---")
    file_count = 0
    folder_count = 0

    if not os.path.exists(start_path):
        print(f"ERROR: The path '{start_path}' does not exist. Please check the path.")
        return

    if not os.path.isdir(start_path):
        print(f"ERROR: The path '{start_path}' is not a directory.")
        return

    for root_directory, sub_directories, files_in_current_directory in os.walk(start_path):
        folder_count += len(sub_directories) # Counts subdirectories encountered
        for filename in files_in_current_directory:
            file_path = os.path.join(root_directory, filename)
            print(file_path)
            file_count += 1

    print(f"\n--- Scan Complete ---")
    print(f"Total folders scanned (including subfolders): Approximately {folder_count} (os.walk yields them progressively)")
    print(f"Total files found: {file_count}")

# --- Main Execution ---
if __name__ == "__main__":
    # Step 1: Mount Google Drive
    try:
        drive.mount('/content/drive', force_remount=True)
        print("Google Drive mounted successfully.")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        # If Drive doesn't mount, we can't proceed.
        # You might want to exit or handle this error more gracefully.
        # For this script, we'll let it try to proceed but it will likely fail if the path is Drive-based.

    # Step 2: Define the starting path in your Google Drive
    # To list ALL files in your "My Drive":
    drive_start_path = "/content/drive/MyDrive/CryptoDataCCXT/"

    # To list files in a SPECIFIC FOLDER within "My Drive":
    # drive_start_path = "/content/drive/MyDrive/YourSpecificFolder/AnotherSubFolder/" # Example

    # **IMPORTANT**: If you changed the name of "My Drive" (e.g., to "My Files"),
    # adjust the path accordingly.
    # Also, ensure the folder path is correct if you are not starting from the root of "My Drive".

    # Check if the path exists after attempting to mount
    if os.path.exists(drive_start_path):
        list_all_files_in_drive(drive_start_path)
    elif drive_start_path.startswith("/content/drive/"): # If it looks like a drive path but doesn't exist
        print(f"\nWARNING: The specified Google Drive path '{drive_start_path}' was not found after attempting to mount.")
        print("Please verify the path and ensure Google Drive is mounted correctly if you intended to use it.")
        print("If you see this after a successful mount message, the specific folder path might be incorrect.")
    else: # If it's not a drive path and doesn't exist
         print(f"\nERROR: The specified path '{drive_start_path}' does not exist.")

In [ ]:
import matplotlib.pyplot as plt
if data is not None:
    plt.figure(figsize=(15, 7))
    data['Close'].plot(label='Close Price', alpha=0.7)
    data[short_sma_col_name].plot(label=short_sma_col_name)
    data[long_sma_col_name].plot(label=long_sma_col_name)
    plt.title(f'Price, {short_sma_col_name}, and {long_sma_col_name}')
    plt.legend()
    plt.show()

In [ ]:
class SmaCross(Strategy):
    def init(self):
        # Access the pre-calculated SMA series from the input DataFrame.
        # 'self.data' provides an interface to the data, making columns accessible.
        # The library will pass the correct series slices to functions like crossover.
        self.sma1 = self.data.df[short_sma_col_name] # Short SMA
        self.sma2 = self.data.df[long_sma_col_name] # Long SMA

        # Optional: You can also use self.I to wrap existing series if you prefer,
        # though direct access as above is fine for pre-calculated columns.
        self.sma1 = self.I(lambda x: x, self.data.df[short_sma_col_name], name="ShortSMA")
        self.sma2 = self.I(lambda x: x, self.data.df[long_sma_col_name], name="LongSMA")

        print(f"DEBUG init(): Short SMA series (first 5 values from data): \n{self.sma1[:5]}") # Use slicing for pandas series
        print(f"DEBUG init(): Long SMA series (first 5 values from data): \n{self.sma2[:5]}")


    def next(self):
        # The `crossover` function takes the two series.
        # It implicitly looks at the current and previous values to determine a cross.

        # If short SMA crosses above long SMA, and we're not already in a position, buy.
        if crossover(self.sma1, self.sma2) and not self.position:
            self.buy()
            # print(f"DEBUG: Buy signal on {self.data.index[-1]}") # Uncomment for signal logging

        # If short SMA crosses below long SMA, and we are in a position, sell/close.
        elif crossover(self.sma2, self.sma1) and self.position:
            self.position.close()
            # print(f"DEBUG: Sell signal on {self.data.index[-1]}") # Uncomment for signal logging


In [ ]:
import pandas as pd
import os
from google.colab import drive
from backtesting import Backtest, Strategy
from backtesting.lib import crossover # A utility for crossover detection

# --- Mount Google Drive ---
try:
    drive.mount('/content/drive', force_remount=False)
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    raise RuntimeError("Google Drive could not be mounted. Script aborted.")

# --- Load Your Data with Indicators ---
GDRIVE_INDICATORS_PATH = "/content/drive/MyDrive/CryptoDataCCXT/with_indicators/"
# Using the daily BTC file that already has indicators
file_to_process = "BTC_USDT_1d_with_indicators.csv"
full_file_path = os.path.join(GDRIVE_INDICATORS_PATH, file_to_process)

print(f"\n--- Loading data for backtesting: {full_file_path} ---")

if not os.path.exists(full_file_path):
    print(f"ERROR: File not found at {full_file_path}. Cannot proceed with backtest.")
else:
    try:
        data = pd.read_csv(full_file_path, index_col='datetime_utc', parse_dates=True)
        print("Successfully loaded the data.")
        print(f"Data shape: {data.shape}")

        # Ensure essential columns are present (backtesting.py needs them uppercase)
        required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        if not all(col in data.columns for col in required_cols):
            print(f"ERROR: DataFrame is missing one or more required OHLCV columns: {required_cols}")
            print(f"Available columns: {data.columns.tolist()}")
            # Consider exiting or raising an error if essential columns are missing
            raise ValueError("DataFrame missing essential OHLCV columns for backtesting.")

        # --- Define Indicator Columns for the Strategy ---
        # These names should match the columns in your CSV file.
        # Based on your master_indicator_configs for BTC:
        # "BTC": { "sma_short": {"window": 10}, "sma_long":  {"window": 50}, ... }
        short_sma_col_name = 'BTC_SMA_10' # Adjust if your sma_short for BTC has a different window
        long_sma_col_name = 'BTC_SMA_50'  # Adjust if your sma_long for BTC has a different window

        if not all(col in data.columns for col in [short_sma_col_name, long_sma_col_name]):
            print(f"ERROR: DataFrame is missing SMA columns: '{short_sma_col_name}' or '{long_sma_col_name}'.")
            print(f"Available columns: {data.columns.tolist()}")
            raise ValueError("DataFrame missing SMA columns for strategy.")


        # --- Run the Backtest ---
        print("\n--- Starting Backtest ---")
        # Initialize Backtest
        # Cash: Initial capital
        # Commission: Brokerage commission per trade (e.g., 0.001 for 0.1%)
        # Exclusive_orders: If True, new orders cancel pending orders. Useful for some strategies.
        bt = Backtest(data, SmaCross, cash=100000, commission=.002, exclusive_orders=True)

        # Run the backtest
        stats = bt.run()

        print("\n--- Backtest Statistics ---")
        print(stats)

        # Print individual trades (optional, can be very long)
        # print("\n--- Trades ---")
        # print(stats['_trades']) # This DataFrame contains details of each trade

        # --- Plot the Results ---
        print("\n--- Plotting Backtest Results (this will open in a new browser tab or display inline in some environments) ---")
        bt.plot()
        # In Colab, the plot might try to open a new tab or might not display directly.
        # If it doesn't display, you can save the plot to a file:
        # bt.plot(filename='sma_cross_backtest.html', open_browser=False)
        # print("Plot saved to sma_cross_backtest.html. You can download and open it.")

    except ValueError as ve:
        print(f"ValueError during data preparation or strategy definition: {ve}")
    except Exception as e:
        print(f"An unexpected error occurred during backtesting: {e}")
        import traceback
        traceback.print_exc()

print("\n--- Backtesting script complete ---")

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import drive

def calculate_max_drawdown(equity_series):
    if equity_series.empty or len(equity_series) < 2:
        return 0.0
    running_max = equity_series.cummax()
    drawdown = (equity_series - running_max) / running_max
    max_drawdown = drawdown.min()
    return max_drawdown if pd.notna(max_drawdown) else 0.0

def simulate_buy_on_signal_sell_on_profit(df_input, initial_capital, short_sma_col_name, long_sma_col_name, commission_rate=0.001):
    df = df_input.copy()

    results = {
        "strategy_applied": "Buy on SMA Crossover, Sell ONLY for Profit (on reverse crossover)",
        "trades": [],
        "initial_capital": initial_capital,
        "final_equity": initial_capital,
        "return_percentage": 0.0,
        "max_drawdown_percentage": 0.0, # Will be calculated on overall equity
        "number_of_trades": 0,
        "winning_trades": 0,
        "status": "Simulation not run yet."
    }

    # Ensure required columns exist
    required_cols = ['Open', 'Close', short_sma_col_name, long_sma_col_name]
    if not all(col in df.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df.columns]
        results["status"] = f"Error: Missing required columns: {missing}"
        print(results["status"])
        return results

    # Generate SMA crossover signals
    df['short_gt_long'] = df[short_sma_col_name] > df[long_sma_col_name]
    df['prev_short_le_long'] = df[short_sma_col_name].shift(1) <= df[long_sma_col_name].shift(1)
    df['buy_trigger_day'] = df['short_gt_long'] & df['prev_short_le_long']

    df['short_lt_long'] = df[short_sma_col_name] < df[long_sma_col_name]
    df['prev_short_ge_long'] = df[short_sma_col_name].shift(1) >= df[long_sma_col_name].shift(1)
    df['potential_sell_trigger_day'] = df['short_lt_long'] & df['prev_short_ge_long']

    current_capital = initial_capital
    position_open = False
    entry_price = 0.0
    shares_held = 0.0
    equity_over_time = pd.Series(index=df.index, dtype=float)
    equity_over_time.iloc[0] = initial_capital # Start with initial capital

    print("\nSimulating trades...")
    for i in range(1, len(df)): # Start from the second day to allow for .shift(1)
        current_date = df.index[i]
        equity_over_time[current_date] = equity_over_time[df.index[i-1]] # Carry forward equity initially

        if position_open:
            # Update equity based on current price of shares held
            equity_over_time[current_date] = shares_held * df['Close'].iloc[i] + (current_capital - (shares_held * entry_price * (1+commission_rate) if i == df.index.get_loc(entry_date)+1 else 0) ) # More precise would be cash + asset value
            equity_over_time[current_date] = shares_held * df['Close'].iloc[i] # Value of asset holding
            # If we also had cash, it would be equity_over_time[current_date] += cash_balance
            # For simplicity in this strategy, assuming all capital is deployed or is in asset.
            # Let's track equity as (shares_held * current_close_price) if in position, else cash.
            # If not using margin, cash = current_capital - (shares_bought * entry_price_with_commission)
            # This part gets complex for precise equity curve if not all capital is used.
            # For simplicity, let's use a common backtesting approach:
            # If in position, equity = shares * current_price. If not, equity = cash.

        # Check for Potential Sell Signal
        if position_open and df['potential_sell_trigger_day'].iloc[i]:
            # Exit only if profitable (current open price > entry price)
            # Sell on next day's open
            potential_exit_price = df['Open'].iloc[i] # Sell on the open of the day signal occurred

            # Check profitability (more accurately, current price vs entry price)
            # For this example, let's check if current Close is profitable to simplify
            # A real exit would use next bar's Open.
            # For this logic: if a sell signal appears, and current *value* > entry *value*, then sell.
            current_close_price = df['Close'].iloc[i] # Price at which sell signal is confirmed

            # More precise: if we were to sell at next open, would it be profitable?
            # For simplicity, let's assume we check profit at the *close* of the signal day,
            # and if profitable, sell at that close.
            if current_close_price > entry_price: # Profitable before commission
                sell_value = shares_held * current_close_price
                sell_value_after_commission = sell_value * (1 - commission_rate)

                profit = sell_value_after_commission - (shares_held * entry_price * (1+commission_rate)) # This is not right
                # Initial cost = shares_held * entry_price (ignoring commission impact on share count for now)
                initial_cost_of_shares = shares_held * entry_price # The price paid per share

                if current_close_price * (1 - commission_rate) > entry_price_with_commission: # Sell if net price > net entry price
                    print(f"{current_date.strftime('%Y-%m-%d')}: Potential Sell Signal. Current Close ({current_close_price:.2f}) > Entry ({entry_price:.2f}). Selling.")
                    current_capital = shares_held * current_close_price * (1 - commission_rate)
                    results["trades"].append({
                        "entry_date": entry_date,
                        "entry_price": entry_price,
                        "exit_date": current_date,
                        "exit_price": current_close_price,
                        "profit_pct": ((current_close_price * (1-commission_rate)) / (entry_price_with_commission) - 1) * 100
                    })
                    if current_close_price > entry_price_with_commission : results["winning_trades"] += 1
                    position_open = False
                    shares_held = 0.0
                    entry_price = 0.0
                    results["number_of_trades"] += 1
                else:
                    print(f"{current_date.strftime('%Y-%m-%d')}: Potential Sell Signal. But Current Close ({current_close_price:.2f}) not profitable vs Entry ({entry_price_with_commission:.2f} incl. commission). Holding.")
            else:
                print(f"{current_date.strftime('%Y-%m-%d')}: Potential Sell Signal. But Current Close ({current_close_price:.2f}) <= Entry ({entry_price:.2f}). Holding due to loss.")

        # Check for Buy Signal (only if not already in a position)
        # Entry on next day's open
        if not position_open and df['buy_trigger_day'].iloc[i-1]: # Signal on previous day's close
            entry_date = current_date # Entry on current day's open
            entry_price = df['Open'].iloc[i]
            entry_price_with_commission = entry_price * (1 + commission_rate) # Effective entry price

            shares_held = current_capital / entry_price_with_commission
            # current_capital -= shares_held * entry_price_with_commission # This assumes all capital deployed
            # For tracking equity if all in:
            equity_over_time[current_date] = shares_held * df['Close'].iloc[i]

            print(f"{entry_date.strftime('%Y-%m-%d')}: Buy Signal. Entering long at {entry_price:.2f}. Shares: {shares_held:.4f}")
            position_open = True

        # Update equity for the day
        if position_open:
            equity_over_time[current_date] = shares_held * df['Close'].iloc[i]
        else:
            equity_over_time[current_date] = current_capital # Cash if not in position

    # If position is still open at the end, liquidate it
    if position_open:
        final_close_price = df['Close'].iloc[-1]
        print(f"End of data. Position still open. Liquidating at {df.index[-1].strftime('%Y-%m-%d')} Close: {final_close_price:.2f}")

        # Only sell if profitable at the very end
        if final_close_price * (1-commission_rate) > entry_price_with_commission:
            current_capital = shares_held * final_close_price * (1 - commission_rate)
            results["trades"].append({
                "entry_date": entry_date,
                "entry_price": entry_price,
                "exit_date": df.index[-1],
                "exit_price": final_close_price,
                "profit_pct": ((final_close_price * (1-commission_rate)) / (entry_price_with_commission) - 1) * 100
            })
            if final_close_price > entry_price_with_commission: results["winning_trades"] += 1
            results["number_of_trades"] += 1
            print("  Sold at profit at end.")
        else:
            # As per strategy "never sell when there is a loss", we might just value the holding.
            # However, for backtesting, we usually liquidate. Let's assume if it's a loss at the end,
            # for the purpose of this strategy, we are still "holding" that loss on paper.
            # The final equity will reflect the value of shares.
            current_capital = shares_held * final_close_price # Value of holding, no sell commission yet if not sold
            print(f"  Position held at a loss at end. Value: {current_capital:.2f} (Entry was {entry_price_with_commission:.2f} net)")
            # If we wanted to record this "unrealized loss" trade for stats:
            # results["trades"].append({ ... profit_pct would be negative ... })
            # results["number_of_trades"] += 1
            # For this strategy, let's say an "exit" only happens on profit. So this doesn't count as a closed trade if at loss.

    results["final_equity"] = equity_over_time.iloc[-1]
    results["return_percentage"] = ((results["final_equity"] - initial_capital) / initial_capital) * 100
    results["max_drawdown_percentage"] = calculate_max_drawdown(equity_over_time) * 100
    results["status"] = "Simulation complete."

    if not results["trades"]:
        results["status"] += " No profitable sell opportunities occurred based on SMA crossover signals."

    return results


# --- Main Execution ---
if __name__ == "__main__":
    try:
        drive.mount('/content/drive', force_remount=True)
        print("Google Drive mounted successfully.")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        raise RuntimeError("Google Drive could not be mounted. Script aborted.")

    GDRIVE_INDICATORS_PATH = "/content/drive/MyDrive/CryptoDataCCXT/with_indicators/"
    file_to_process = "BTC_USDT_1d_with_indicators.csv"
    full_file_path = os.path.join(GDRIVE_INDICATORS_PATH, file_to_process)

    SHORT_SMA_COL_NAME = 'BTC_SMA_10'
    LONG_SMA_COL_NAME = 'BTC_SMA_50'
    INITIAL_CAPITAL = 100000.0
    COMMISSION_RATE = 0.001

    print(f"\n--- Simulating 'Buy on SMA Crossover, Sell ONLY for Profit' for: {file_to_process} ---")

    if not os.path.exists(full_file_path):
        print(f"ERROR: File not found at {full_file_path}. Cannot proceed.")
    else:
        try:
            df_loaded = pd.read_csv(full_file_path, index_col='datetime_utc', parse_dates=True)
            print(f"Successfully loaded data. Shape: {df_loaded.shape}")

            if df_loaded.empty:
                print("ERROR: Loaded DataFrame is empty.")
            else:
                strategy_performance = simulate_buy_on_signal_sell_on_profit(
                    df_loaded,
                    INITIAL_CAPITAL,
                    SHORT_SMA_COL_NAME,
                    LONG_SMA_COL_NAME,
                    COMMISSION_RATE
                )

                print("\n--- Strategy Performance ---")
                for key, value in strategy_performance.items():
                    if key == "trades":
                        print(f"  {key.replace('_', ' ').title()}:")
                        if value:
                            for i, trade in enumerate(value):
                                print(f"    Trade {i+1}: Entry {trade['entry_date'].strftime('%Y-%m-%d')} @ {trade['entry_price']:.2f} | Exit {trade['exit_date'].strftime('%Y-%m-%d')} @ {trade['exit_price']:.2f} | Profit: {trade['profit_pct']:.2f}%")
                        else:
                            print("    No trades executed that met profit criteria.")
                    elif isinstance(value, float) and ("percentage" in key or "equity" in key or "price" in key or "capital" in key):
                        print(f"  {key.replace('_', ' ').title()}: {value:,.2f}")
                    else:
                        print(f"  {key.replace('_', ' ').title()}: {value}")
        except Exception as e:
            print(f"An error occurred during the simulation: {e}")
            import traceback
            traceback.print_exc()
    print("\n--- Script complete ---")

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import drive

def calculate_max_drawdown(equity_series):
    """
    Calculates the maximum drawdown from an equity series.
    Args:
        equity_series (pd.Series): A pandas Series representing the equity over time.
    Returns:
        float: The maximum drawdown as a negative percentage (e.g., -0.25 for -25%).
               Returns 0.0 if no drawdown or insufficient data.
    """
    if equity_series.empty or len(equity_series) < 2:
        return 0.0
    running_max = equity_series.cummax()
    drawdown = (equity_series - running_max) / running_max
    max_drawdown = drawdown.min()
    return max_drawdown if pd.notna(max_drawdown) else 0.0

def simulate_daily_dca_and_hold(df_input, daily_investment_amount, commission_rate=0.001):
    """
    Simulates a daily Dollar-Cost Averaging (DCA) and Hold strategy.
    """
    df = df_input.copy()

    results = {
        "strategy_applied": "Daily Dollar-Cost Averaging and Hold",
        "daily_investment_amount": daily_investment_amount,
        "total_days": len(df),
        "total_cash_invested": 0.0,
        "total_units_held_final": 0.0,
        "average_cost_per_unit": 0.0,
        "final_portfolio_value_before_sell_commission": 0.0,
        "final_portfolio_value_after_sell_commission": 0.0,
        "net_profit": 0.0,
        "return_on_investment_percentage": 0.0,
        "max_drawdown_percentage_on_portfolio_value": 0.0,
        "status": "Simulation not run yet."
    }

    required_cols = ['Open', 'Close']
    if not all(col in df.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df.columns]
        results["status"] = f"Error: Missing required columns: {missing}"
        print(results["status"])
        return results

    if df.empty:
        results["status"] = "Error: DataFrame is empty."
        print(results["status"])
        return results

    total_units_accumulated = 0.0
    total_cash_invested = 0.0
    portfolio_value_over_time = pd.Series(index=df.index, dtype=float)

    print("\nSimulating daily DCA purchases...")
    for i in range(len(df)):
        current_date = df.index[i]
        purchase_price = df['Open'].iloc[i] # Buy at Open

        # Calculate units bought today
        cash_for_purchase_after_commission = daily_investment_amount * (1 - commission_rate)
        units_bought_today = cash_for_purchase_after_commission / purchase_price

        total_units_accumulated += units_bought_today
        total_cash_invested += daily_investment_amount # This is the actual cash outlaid

        # Portfolio value at the end of this day (using Close price)
        current_portfolio_value = total_units_accumulated * df['Close'].iloc[i]
        portfolio_value_over_time[current_date] = current_portfolio_value

        if (i + 1) % 365 == 0: # Print progress yearly
            print(f"Year { (i+1)//365 }: Invested ${total_cash_invested:,.2f}, Holding {total_units_accumulated:.4f} units, Value ${current_portfolio_value:,.2f}")

    results["total_cash_invested"] = total_cash_invested
    results["total_units_held_final"] = total_units_accumulated

    if total_units_accumulated > 0:
        results["average_cost_per_unit"] = total_cash_invested / total_units_accumulated

    # --- Final Liquidation at the end of the period ---
    final_close_price = df['Close'].iloc[-1]
    results["final_portfolio_value_before_sell_commission"] = total_units_accumulated * final_close_price
    results["final_portfolio_value_after_sell_commission"] = results["final_portfolio_value_before_sell_commission"] * (1 - commission_rate)

    results["net_profit"] = results["final_portfolio_value_after_sell_commission"] - total_cash_invested

    if total_cash_invested > 0:
        results["return_on_investment_percentage"] = (results["net_profit"] / total_cash_invested) * 100

    if not portfolio_value_over_time.empty:
        results["max_drawdown_percentage_on_portfolio_value"] = calculate_max_drawdown(portfolio_value_over_time) * 100

    results["status"] = f"Simulation complete. Held until {df.index[-1].strftime('%Y-%m-%d')}."
    print(results["status"])
    return results

# --- Main Execution ---
if __name__ == "__main__":
    try:
        drive.mount('/content/drive', force_remount=True)
        print("Google Drive mounted successfully.")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        raise RuntimeError("Google Drive could not be mounted. Script aborted.")

    GDRIVE_INDICATORS_PATH = "/content/drive/MyDrive/CryptoDataCCXT/with_indicators/"
    # Using the file that already has indicators, but DCA only needs OHLC.
    # We could also use the base CCXT output file (e.g., BTC_USDT_1d_4years.csv)
    # This file should at least contain 'Open' and 'Close' columns with a DatetimeIndex.
    file_to_process = "BTC_USDT_1d_with_indicators.csv"
    full_file_path = os.path.join(GDRIVE_INDICATORS_PATH, file_to_process)

    DAILY_INVESTMENT = 68.50 # To get approx $100k invested over ~4 years
    COMMISSION_RATE = 0.001 # 0.1%

    print(f"\n--- Simulating 'Daily DCA and Hold' for: {file_to_process} ---")
    print(f"Daily Investment: ${DAILY_INVESTMENT:.2f}")

    if not os.path.exists(full_file_path):
        print(f"ERROR: File not found at {full_file_path}. Cannot proceed.")
    else:
        try:
            df_loaded = pd.read_csv(full_file_path, index_col='datetime_utc', parse_dates=True)
            # If using the _with_indicators.csv, Open and Close are already uppercase.
            # If using base CCXT output, ensure 'Open' and 'Close' columns exist or rename them.
            # For this simulation, we assume 'Open' and 'Close' columns are present as needed.

            print(f"Successfully loaded data. Shape: {df_loaded.shape}. Data from {df_loaded.index.min()} to {df_loaded.index.max()}")

            if df_loaded.empty:
                print("ERROR: Loaded DataFrame is empty.")
            else:
                # Ensure 'Open' and 'Close' columns are available (they are in uppercase in _with_indicators files)
                if not {'Open', 'Close'}.issubset(df_loaded.columns):
                    # Attempt to rename if lowercase ohlc are present from a base file
                    rename_map_dca = {'open': 'Open', 'close': 'Close'}
                    cols_to_rename_dca = {k: v for k, v in rename_map_dca.items() if k in df_loaded.columns}
                    if 'Open' not in df_loaded.columns and 'open' in cols_to_rename_dca : df_loaded.rename(columns={'open':'Open'}, inplace=True)
                    if 'Close' not in df_loaded.columns and 'close' in cols_to_rename_dca : df_loaded.rename(columns={'close':'Close'}, inplace=True)

                    if not {'Open', 'Close'}.issubset(df_loaded.columns):
                        print(f"ERROR: DataFrame must contain 'Open' and 'Close' columns. Found: {df_loaded.columns.tolist()}")
                        raise ValueError("Missing essential OHLC columns for DCA simulation.")


                dca_performance = simulate_daily_dca_and_hold(
                    df_loaded,
                    DAILY_INVESTMENT,
                    COMMISSION_RATE
                )

                print("\n--- Daily DCA and Hold Strategy Performance ---")
                for key, value in dca_performance.items():
                    if isinstance(value, float) and ("percentage" in key or "value" in key or "amount" in key or "cost" in key or "profit" in key or "invested" in key):
                        print(f"  {key.replace('_', ' ').title()}: {value:,.2f}")
                    else:
                        print(f"  {key.replace('_', ' ').title()}: {value}")

        except Exception as e:
            print(f"An error occurred during the simulation: {e}")
            import traceback
            traceback.print_exc()
    print("\n--- Script complete ---")

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import drive

# --- Assume your indicator functions (add_sma, etc.) are defined ---
# --- Assume your master_indicator_configs is defined ---

def calculate_max_drawdown(equity_series):
    if equity_series.empty or len(equity_series) < 2: return 0.0
    running_max = equity_series.cummax()
    drawdown = (equity_series - running_max) / running_max
    max_drawdown = drawdown.min()
    return max_drawdown if pd.notna(max_drawdown) else 0.0

def simulate_signal_triggered_daily_dca(
    df_input,
    daily_allocation,
    short_sma_col_name, # For SMA Crossover signal
    long_sma_col_name,  # For SMA Crossover signal
    commission_rate=0.001,
    deploy_all_accumulated_cash_on_signal=True # If True, invests all waiting cash on signal
):
    df = df_input.copy()
    results = {
        "strategy_applied": "Signal-Triggered Daily DCA (SMA Crossover Entry, Hold)",
        "daily_allocation": daily_allocation,
        "total_days_processed": len(df),
        "total_cash_allocated_over_period": len(df) * daily_allocation,
        "total_cash_actually_invested": 0.0,
        "remaining_cash_at_end_if_not_all_invested": 0.0,
        "total_units_held_final": 0.0,
        "average_cost_per_unit_of_invested_cash": 0.0,
        "final_portfolio_value_before_sell_commission": 0.0,
        "final_portfolio_value_after_sell_commission": 0.0,
        "net_profit_on_invested_cash": 0.0,
        "return_on_invested_cash_percentage": 0.0,
        "return_on_total_allocated_cash_percentage": 0.0,
        "max_drawdown_percentage_on_portfolio_value": 0.0,
        "number_of_buy_transactions": 0,
        "buy_days": [],
        "status": "Simulation not run yet."
    }

    required_cols = ['Open', 'Close', short_sma_col_name, long_sma_col_name]
    if not all(col in df.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df.columns]
        results["status"] = f"Error: Missing required columns: {missing}"
        print(results["status"])
        return results

    # --- Generate Entry Signals (SMA Crossover) ---
    df['short_gt_long'] = df[short_sma_col_name] > df[long_sma_col_name]
    df['prev_short_le_long'] = df[short_sma_col_name].shift(1) <= df[long_sma_col_name].shift(1)
    df['buy_signal_on_prev_close'] = df['short_gt_long'] & df['prev_short_le_long']
    # We act on signal_on_prev_close by buying at current day's Open

    accumulated_cash_waiting = 0.0
    total_units_held = 0.0
    total_cash_invested_into_asset = 0.0 # Tracks only cash that actually bought the asset

    portfolio_value_over_time = pd.Series(index=df.index, dtype=float)
    portfolio_value_over_time.iloc[0] = 0 # Start with 0 asset value, cash is separate

    print("\nSimulating Signal-Triggered Daily DCA...")
    for i in range(len(df)):
        current_date = df.index[i]
        accumulated_cash_waiting += daily_allocation # Add today's allocation

        buy_signal_active_today = df['buy_signal_on_prev_close'].iloc[i] if i > 0 else False # Signal based on previous day's close

        if buy_signal_active_today:
            cash_to_invest_this_time = 0
            if deploy_all_accumulated_cash_on_signal:
                cash_to_invest_this_time = accumulated_cash_waiting
            else: # Only invest current day's allocation on signal
                cash_to_invest_this_time = daily_allocation
                # The rest of accumulated_cash_waiting remains if this logic is chosen
                # This part needs refinement if not deploying all. Let's stick to deploy_all_accumulated_cash_on_signal=True for now.

            if cash_to_invest_this_time > 0:
                purchase_price = df['Open'].iloc[i]
                purchase_amount_after_commission = cash_to_invest_this_time * (1 - commission_rate)
                units_bought_today = purchase_amount_after_commission / purchase_price

                total_units_held += units_bought_today
                total_cash_invested_into_asset += cash_to_invest_this_time # The amount taken from accumulated_cash_waiting
                accumulated_cash_waiting -= cash_to_invest_this_time # Deduct invested amount

                results["number_of_buy_transactions"] += 1
                results["buy_days"].append(current_date)
                print(f"{current_date.strftime('%Y-%m-%d')}: BUY SIGNAL. Invested ${cash_to_invest_this_time:,.2f} at ${purchase_price:,.2f}. Units: {units_bought_today:.6f}. Total units: {total_units_held:.6f}")

        # Portfolio value at the end of this day
        current_portfolio_value = total_units_held * df['Close'].iloc[i]
        portfolio_value_over_time[current_date] = current_portfolio_value + accumulated_cash_waiting # Total net worth

    results["total_cash_actually_invested"] = total_cash_invested_into_asset
    results["remaining_cash_at_end_if_not_all_invested"] = accumulated_cash_waiting
    results["total_units_held_final"] = total_units_held

    if total_units_held > 0 and total_cash_invested_into_asset > 0:
        results["average_cost_per_unit_of_invested_cash"] = total_cash_invested_into_asset / total_units_held

    final_asset_value = total_units_held * df['Close'].iloc[-1]
    results["final_portfolio_value_before_sell_commission"] = final_asset_value + accumulated_cash_waiting

    # Simulate selling all held assets at the end
    final_asset_value_after_sell_commission = final_asset_value * (1 - commission_rate)
    results["final_portfolio_value_after_sell_commission"] = final_asset_value_after_sell_commission + accumulated_cash_waiting

    results["net_profit_on_invested_cash"] = (final_asset_value_after_sell_commission - total_cash_invested_into_asset)

    if total_cash_invested_into_asset > 0:
        results["return_on_invested_cash_percentage"] = (results["net_profit_on_invested_cash"] / total_cash_invested_into_asset) * 100

    # ROI on total allocated cash (includes cash never invested if signals were rare)
    net_profit_overall = results["final_portfolio_value_after_sell_commission"] - results["total_cash_allocated_over_period"]
    if results["total_cash_allocated_over_period"] > 0:
        results["return_on_total_allocated_cash_percentage"] = (net_profit_overall / results["total_cash_allocated_over_period"]) * 100

    if not portfolio_value_over_time.empty: # portfolio_value_over_time tracks asset value + waiting cash
        results["max_drawdown_percentage_on_portfolio_value"] = calculate_max_drawdown(portfolio_value_over_time) * 100

    results["status"] = f"Simulation complete. Held until {df.index[-1].strftime('%Y-%m-%d')}."
    print(results["status"])
    return results

# --- Main Execution ---
if __name__ == "__main__":
    try:
        drive.mount('/content/drive', force_remount=True)
        print("Google Drive mounted successfully.")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        raise RuntimeError("Google Drive could not be mounted. Script aborted.")

    GDRIVE_INDICATORS_PATH = "/content/drive/MyDrive/CryptoDataCCXT/with_indicators/"
    file_to_process = "BTC_USDT_1d_with_indicators.csv"
    full_file_path = os.path.join(GDRIVE_INDICATORS_PATH, file_to_process)

    DAILY_ALLOCATION = 200.00
    SHORT_SMA_COL_NAME = 'BTC_SMA_10' # From your BTC master_config
    LONG_SMA_COL_NAME = 'BTC_SMA_50'  # From your BTC master_config
    COMMISSION_RATE = 0.001

    print(f"\n--- Simulating 'Signal-Triggered Daily DCA' for: {file_to_process} ---")
    print(f"Daily Allocation: ${DAILY_ALLOCATION:.2f}")

    if not os.path.exists(full_file_path):
        print(f"ERROR: File not found at {full_file_path}. Cannot proceed.")
    else:
        try:
            df_loaded = pd.read_csv(full_file_path, index_col='datetime_utc', parse_dates=True)
            print(f"Successfully loaded data. Shape: {df_loaded.shape}. Data from {df_loaded.index.min()} to {df_loaded.index.max()}")

            if df_loaded.empty:
                print("ERROR: Loaded DataFrame is empty.")
            else:
                # Ensure necessary columns exist
                required_for_sim = ['Open', 'Close', SHORT_SMA_COL_NAME, LONG_SMA_COL_NAME]
                if not all(col in df_loaded.columns for col in required_for_sim):
                    raise ValueError(f"DataFrame missing one or more required columns for simulation: {required_for_sim}. Found: {df_loaded.columns.tolist()}")

                dca_signal_performance = simulate_signal_triggered_daily_dca(
                    df_loaded,
                    DAILY_ALLOCATION,
                    SHORT_SMA_COL_NAME,
                    LONG_SMA_COL_NAME,
                    COMMISSION_RATE,
                    deploy_all_accumulated_cash_on_signal=True # Key parameter
                )

                print("\n--- Signal-Triggered Daily DCA Performance ---")
                for key, value in dca_signal_performance.items():
                    if key == "buy_days":
                        print(f"  {key.replace('_', ' ').title()}: {len(value)} days. First 5: {[d.strftime('%Y-%m-%d') for d in value[:5]]}")
                    elif isinstance(value, float) and ("percentage" in key or "value" in key or "amount" in key or "cost" in key or "profit" in key or "invested" in key or "allocation" in key):
                        print(f"  {key.replace('_', ' ').title()}: {value:,.2f}")
                    else:
                        print(f"  {key.replace('_', ' ').title()}: {value}")
        except Exception as e:
            print(f"An error occurred during the simulation: {e}")
            import traceback
            traceback.print_exc()
    print("\n--- Script complete ---")

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import drive

def calculate_max_drawdown(equity_series):
    if equity_series.empty or len(equity_series) < 2: return 0.0
    running_max = equity_series.cummax()
    drawdown = (equity_series - running_max) / running_max
    max_drawdown = drawdown.min()
    return max_drawdown if pd.notna(max_drawdown) else 0.0

def simulate_dca_with_signal_logging(
    df_input,
    daily_investment_amount,
    short_sma_col_name, # For SMA Crossover signal logging
    long_sma_col_name,  # For SMA Crossover signal logging
    commission_rate=0.001
):
    df = df_input.copy()
    results = {
        "strategy_applied": "Daily DCA with SMA Crossover Signal Logging",
        "daily_investment_amount": daily_investment_amount,
        "total_days": len(df),
        "total_cash_invested": 0.0,
        "total_units_held_final": 0.0,
        "average_cost_per_unit": 0.0,
        "final_portfolio_value_before_sell_commission": 0.0,
        "final_portfolio_value_after_sell_commission": 0.0,
        "net_profit": 0.0,
        "return_on_investment_percentage": 0.0,
        "max_drawdown_percentage_on_portfolio_value": 0.0,
        "days_with_buy_signal_when_investing": 0,
        "signal_days_details": [],
        "status": "Simulation not run yet."
    }

    required_cols = ['Open', 'Close', short_sma_col_name, long_sma_col_name]
    if not all(col in df.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df.columns]
        results["status"] = f"Error: Missing required columns: {missing}"
        print(results["status"])
        return results

    if df.empty:
        results["status"] = "Error: DataFrame is empty."
        print(results["status"])
        return results

    # --- Generate Entry Signals (for logging purposes) ---
    df['short_gt_long'] = df[short_sma_col_name] > df[long_sma_col_name]
    df['prev_short_le_long'] = df[short_sma_col_name].shift(1) <= df[long_sma_col_name].shift(1)
    df['buy_signal_on_prev_close'] = df['short_gt_long'] & df['prev_short_le_long']

    total_units_accumulated = 0.0
    total_cash_invested = 0.0
    portfolio_value_over_time = pd.Series(index=df.index, dtype=float)
    portfolio_value_over_time.iloc[0] = 0 # Start with 0 asset value

    print("\nSimulating Daily DCA with signal logging...")
    for i in range(len(df)):
        current_date = df.index[i]
        purchase_price = df['Open'].iloc[i] # Buy at Open

        cash_for_purchase_after_commission = daily_investment_amount * (1 - commission_rate)
        units_bought_today = cash_for_purchase_after_commission / purchase_price

        total_units_accumulated += units_bought_today
        total_cash_invested += daily_investment_amount

        current_portfolio_value = total_units_accumulated * df['Close'].iloc[i]
        portfolio_value_over_time[current_date] = current_portfolio_value

        # Log if a buy signal was active when this DCA purchase was made
        signal_active_for_this_entry = df['buy_signal_on_prev_close'].iloc[i] if i > 0 else False
        if signal_active_for_this_entry:
            results["days_with_buy_signal_when_investing"] += 1
            results["signal_days_details"].append({
                "date": current_date,
                "price_bought": purchase_price,
                "units_bought": units_bought_today
            })
            # print(f"{current_date.strftime('%Y-%m-%d')}: DCA Purchase. BUY SIGNAL was active. Price: ${purchase_price:,.2f}")

        if (i + 1) % 365 == 0: # Print progress yearly
            print(f"Year { (i+1)//365 }: Invested ${total_cash_invested:,.2f}, Holding {total_units_accumulated:.4f} units, Value ${current_portfolio_value:,.2f}. Signal days so far: {results['days_with_buy_signal_when_investing']}")

    results["total_cash_invested"] = total_cash_invested
    results["total_units_held_final"] = total_units_accumulated

    if total_units_accumulated > 0:
        results["average_cost_per_unit"] = total_cash_invested / total_units_accumulated

    final_close_price = df['Close'].iloc[-1]
    results["final_portfolio_value_before_sell_commission"] = total_units_accumulated * final_close_price
    results["final_portfolio_value_after_sell_commission"] = results["final_portfolio_value_before_sell_commission"] * (1 - commission_rate)

    results["net_profit"] = results["final_portfolio_value_after_sell_commission"] - total_cash_invested

    if total_cash_invested > 0:
        results["return_on_investment_percentage"] = (results["net_profit"] / total_cash_invested) * 100

    if not portfolio_value_over_time.empty:
        results["max_drawdown_percentage_on_portfolio_value"] = calculate_max_drawdown(portfolio_value_over_time) * 100

    results["status"] = f"Simulation complete. Held until {df.index[-1].strftime('%Y-%m-%d')}."
    print(results["status"])
    return results

# --- Main Execution ---
if __name__ == "__main__":
    try:
        drive.mount('/content/drive', force_remount=True)
        print("Google Drive mounted successfully.")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        raise RuntimeError("Google Drive could not be mounted. Script aborted.")

    GDRIVE_INDICATORS_PATH = "/content/drive/MyDrive/CryptoDataCCXT/with_indicators/"
    file_to_process = "BTC_USDT_1d_with_indicators.csv"
    full_file_path = os.path.join(GDRIVE_INDICATORS_PATH, file_to_process)

    DAILY_INVESTMENT_OPT_B = 200.00
    SHORT_SMA_COL_NAME = 'BTC_SMA_10'
    LONG_SMA_COL_NAME = 'BTC_SMA_50'
    COMMISSION_RATE = 0.001

    print(f"\n--- Simulating Option B: 'Daily DCA ($200) with SMA Crossover Signal Logging' for: {file_to_process} ---")

    if not os.path.exists(full_file_path):
        print(f"ERROR: File not found at {full_file_path}. Cannot proceed.")
    else:
        try:
            df_loaded = pd.read_csv(full_file_path, index_col='datetime_utc', parse_dates=True)
            print(f"Successfully loaded data. Shape: {df_loaded.shape}. Data from {df_loaded.index.min()} to {df_loaded.index.max()}")

            if df_loaded.empty:
                print("ERROR: Loaded DataFrame is empty.")
            else:
                if not {'Open', 'Close', SHORT_SMA_COL_NAME, LONG_SMA_COL_NAME}.issubset(df_loaded.columns):
                    raise ValueError(f"DataFrame missing one or more required columns. Needed: Open, Close, {SHORT_SMA_COL_NAME}, {LONG_SMA_COL_NAME}. Found: {df_loaded.columns.tolist()}")

                dca_option_b_performance = simulate_dca_with_signal_logging(
                    df_loaded,
                    DAILY_INVESTMENT_OPT_B,
                    SHORT_SMA_COL_NAME,
                    LONG_SMA_COL_NAME,
                    COMMISSION_RATE
                )

                print("\n--- Option B: Daily DCA with Signal Logging Performance ---")
                for key, value in dca_option_b_performance.items():
                    if key == "signal_days_details":
                        print(f"  {key.replace('_', ' ').title()}: First 5 signal-coincident purchases:")
                        for item in value[:5]:
                            print(f"    Date: {item['date'].strftime('%Y-%m-%d')}, Price: {item['price_bought']:.2f}, Units: {item['units_bought']:.6f}")
                    elif isinstance(value, float) and ("percentage" in key or "value" in key or "amount" in key or "cost" in key or "profit" in key or "invested" in key):
                        print(f"  {key.replace('_', ' ').title()}: {value:,.2f}")
                    else:
                        print(f"  {key.replace('_', ' ').title()}: {value}")
        except Exception as e:
            print(f"An error occurred during the simulation: {e}")
            import traceback
            traceback.print_exc()
    print("\n--- Script complete ---")

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import drive

# --- Assume your indicator functions (add_sma, etc.) are defined ---
# --- Assume your master_indicator_configs is defined ---

def calculate_max_drawdown(equity_series):
    if equity_series.empty or len(equity_series) < 2: return 0.0
    running_max = equity_series.cummax()
    drawdown = (equity_series - running_max) / running_max
    max_drawdown = drawdown.min()
    return max_drawdown if pd.notna(max_drawdown) else 0.0

def simulate_option_c_dca(
    df_primary_asset, # DataFrame for BTC with its indicators
    daily_allocation,
    primary_asset_symbol, # e.g., "BTC"
    short_sma_col_name,
    long_sma_col_name,
    commission_rate_primary=0.001,
    # For simplicity, let's assume uninvested cash doesn't earn interest in this example,
    # but one could add a small daily interest accrual to 'cash_pool'.
    # alternative_asset_daily_interest_rate=0.00005 # e.g., ~1.8% APR / 365
    deploy_all_accumulated_cash_on_signal=True
):
    df = df_primary_asset.copy()
    results = {
        "strategy_applied": "Option C: Daily Allocation, Buy Primary on Signal, Else Hold Cash",
        "daily_allocation": daily_allocation,
        "total_days_processed": len(df),
        "total_cash_allocated_over_period": len(df) * daily_allocation,
        "total_cash_invested_in_primary": 0.0,
        "cash_pool_at_end": 0.0,
        "total_primary_asset_units_final": 0.0,
        "average_cost_per_primary_unit": 0.0,
        "final_portfolio_value_before_sell_commission": 0.0, # Value of primary asset + cash pool
        "final_portfolio_value_after_sell_commission": 0.0,
        "net_profit_overall": 0.0,
        "return_on_total_allocated_cash_percentage": 0.0,
        "max_drawdown_percentage_on_portfolio_value": 0.0,
        "number_of_primary_asset_buys": 0,
        "buy_days_primary_asset": [],
        "status": "Simulation not run yet."
    }

    required_cols = ['Open', 'Close', short_sma_col_name, long_sma_col_name]
    if not all(col in df.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df.columns]
        results["status"] = f"Error: Missing required columns in primary asset DataFrame: {missing}"
        print(results["status"])
        return results

    # --- Generate Entry Signals for Primary Asset (e.g., BTC SMA Crossover) ---
    df['short_gt_long'] = df[short_sma_col_name] > df[long_sma_col_name]
    df['prev_short_le_long'] = df[short_sma_col_name].shift(1) <= df[long_sma_col_name].shift(1)
    df['buy_signal_on_prev_close_primary'] = df['short_gt_long'] & df['prev_short_le_long']

    cash_pool = 0.0 # Cash waiting to be invested in primary asset
    primary_asset_units_held = 0.0
    total_cash_invested_in_primary_asset = 0.0

    portfolio_value_over_time = pd.Series(index=df.index, dtype=float)
    portfolio_value_over_time.iloc[0] = daily_allocation # First day's allocation becomes initial cash

    print("\nSimulating Option C: Signal-Triggered DCA for Primary, Cash Accumulation Else...")
    for i in range(len(df)):
        current_date = df.index[i]
        cash_pool += daily_allocation # Add today's allocation to the pool

        # If not the first day, carry forward previous day's asset value for portfolio tracking before today's action
        current_primary_asset_value = primary_asset_units_held * df['Close'].iloc[i-1] if i > 0 else 0
        portfolio_value_over_time[current_date] = current_primary_asset_value + cash_pool


        buy_signal_active_today_primary = df['buy_signal_on_prev_close_primary'].iloc[i] if i > 0 else False

        if buy_signal_active_today_primary:
            cash_to_invest_this_time = 0
            if deploy_all_accumulated_cash_on_signal:
                cash_to_invest_this_time = cash_pool # Invest all available cash
            else: # Only invest current day's allocation (more complex cash pool management)
                cash_to_invest_this_time = min(daily_allocation, cash_pool)

            if cash_to_invest_this_time > 1: # Check if there's meaningful cash to invest
                purchase_price_primary = df['Open'].iloc[i]
                purchase_amount_after_commission = cash_to_invest_this_time * (1 - commission_rate_primary)
                units_bought_today_primary = purchase_amount_after_commission / purchase_price_primary

                primary_asset_units_held += units_bought_today_primary
                total_cash_invested_in_primary_asset += cash_to_invest_this_time
                cash_pool -= cash_to_invest_this_time # Deduct invested amount

                results["number_of_primary_asset_buys"] += 1
                results["buy_days_primary_asset"].append(current_date)
                print(f"{current_date.strftime('%Y-%m-%d')}: PRIMARY ASSET ({primary_asset_symbol}) BUY SIGNAL. Invested ${cash_to_invest_this_time:,.2f} at ${purchase_price_primary:,.2f}. Units: {units_bought_today_primary:.6f}. Cash pool: ${cash_pool:,.2f}")

        # Update portfolio value at end of day
        current_primary_asset_value = primary_asset_units_held * df['Close'].iloc[i]
        portfolio_value_over_time[current_date] = current_primary_asset_value + cash_pool

        if (i + 1) % 365 == 0:
             print(f"Year {(i+1)//365}: Primary Units: {primary_asset_units_held:.4f}, Cash Pool: ${cash_pool:,.2f}, Portfolio Value: ${portfolio_value_over_time[current_date]:,.2f}")


    results["total_cash_actually_invested_in_primary"] = total_cash_invested_in_primary_asset
    results["cash_pool_at_end"] = cash_pool
    results["total_primary_asset_units_final"] = primary_asset_units_held

    if primary_asset_units_held > 0 and total_cash_invested_in_primary_asset > 0:
        results["average_cost_per_primary_unit"] = total_cash_invested_in_primary_asset / primary_asset_units_held

    # Final portfolio value = value of primary asset held + remaining cash pool
    final_primary_asset_value = primary_asset_units_held * df['Close'].iloc[-1]
    results["final_portfolio_value_before_sell_commission"] = final_primary_asset_value + cash_pool

    # Simulate selling all held primary assets at the end
    final_primary_asset_value_after_sell_commission = final_primary_asset_value * (1 - commission_rate_primary)
    results["final_portfolio_value_after_sell_commission"] = final_primary_asset_value_after_sell_commission + cash_pool # Add back cash pool

    # Net profit based on total cash allocated over the period
    results["net_profit_overall"] = results["final_portfolio_value_after_sell_commission"] - results["total_cash_allocated_over_period"]

    if results["total_cash_allocated_over_period"] > 0:
        results["return_on_total_allocated_cash_percentage"] = (results["net_profit_overall"] / results["total_cash_allocated_over_period"]) * 100

    if not portfolio_value_over_time.empty:
        results["max_drawdown_percentage_on_portfolio_value"] = calculate_max_drawdown(portfolio_value_over_time) * 100

    results["status"] = f"Simulation complete. Held until {df.index[-1].strftime('%Y-%m-%d')}."
    print(results["status"])
    return results

# --- Main Execution ---
if __name__ == "__main__":
    try:
        drive.mount('/content/drive', force_remount=True)
        print("Google Drive mounted successfully.")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        raise RuntimeError("Google Drive could not be mounted. Script aborted.")

    GDRIVE_INDICATORS_PATH = "/content/drive/MyDrive/CryptoDataCCXT/with_indicators/"
    file_to_process_btc = "BTC_USDT_1d_with_indicators.csv" # Primary asset data
    full_file_path_btc = os.path.join(GDRIVE_INDICATORS_PATH, file_to_process_btc)

    DAILY_ALLOCATION_OPT_C = 200.00
    # For BTC (Primary Asset)
    PRIMARY_ASSET_SYMBOL = "BTC"
    BTC_SHORT_SMA_COL_NAME = 'BTC_SMA_10'
    BTC_LONG_SMA_COL_NAME = 'BTC_SMA_50'
    COMMISSION_RATE = 0.001

    print(f"\n--- Simulating Option C for Primary Asset: {PRIMARY_ASSET_SYMBOL} from file: {file_to_process_btc} ---")
    print(f"Daily Allocation: ${DAILY_ALLOCATION_OPT_C:.2f}")

    if not os.path.exists(full_file_path_btc):
        print(f"ERROR: Primary asset data file not found at {full_file_path_btc}. Cannot proceed.")
    else:
        try:
            df_btc_loaded = pd.read_csv(full_file_path_btc, index_col='datetime_utc', parse_dates=True)
            print(f"Successfully loaded BTC data. Shape: {df_btc_loaded.shape}. Data from {df_btc_loaded.index.min()} to {df_btc_loaded.index.max()}")

            if df_btc_loaded.empty:
                print("ERROR: Loaded BTC DataFrame is empty.")
            else:
                required_for_sim_btc = ['Open', 'Close', BTC_SHORT_SMA_COL_NAME, BTC_LONG_SMA_COL_NAME]
                if not all(col in df_btc_loaded.columns for col in required_for_sim_btc):
                    raise ValueError(f"BTC DataFrame missing one or more required columns for simulation: {required_for_sim_btc}. Found: {df_btc_loaded.columns.tolist()}")

                option_c_performance = simulate_option_c_dca(
                    df_btc_loaded,
                    DAILY_ALLOCATION_OPT_C,
                    PRIMARY_ASSET_SYMBOL,
                    BTC_SHORT_SMA_COL_NAME,
                    BTC_LONG_SMA_COL_NAME,
                    COMMISSION_RATE,
                    deploy_all_accumulated_cash_on_signal=True
                )

                print("\n--- Option C: Signal-Triggered DCA for Primary, Cash Accumulation Else ---")
                for key, value in option_c_performance.items():
                    if key == "buy_days_primary_asset":
                        print(f"  {key.replace('_', ' ').title()}: {len(value)} days. First 5: {[d.strftime('%Y-%m-%d') for d in value[:5]]}")
                    elif isinstance(value, float) and any(sub in key for sub in ["percentage", "value", "amount", "cost", "profit", "invested", "allocation", "pool"]):
                        print(f"  {key.replace('_', ' ').title()}: {value:,.2f}")
                    else:
                        print(f"  {key.replace('_', ' ').title()}: {value}")
        except Exception as e:
            print(f"An error occurred during the simulation: {e}")
            import traceback
            traceback.print_exc()
    print("\n--- Script complete ---")

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import drive

# --- Step 1: Mount Google Drive ---
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    raise RuntimeError("Google Drive could not be mounted. Script aborted.")

# --- Step 2: Define Master Indicator Configs (or ensure it's loaded from a previous cell) ---
# IMPORTANT: This script assumes 'master_indicator_configs' is already defined in your notebook.
# For this example, we'll use a placeholder to dynamically get column names.
# Replace this with your actual master_indicator_configs dictionary.
master_indicator_configs = {
    "BTC": {
        "ema_short": {"span": 12, "column": "Close"}, # Example: Fast EMA
        "ema_long":  {"span": 26, "column": "Close"}, # Example: Slow EMA
        "rsi":       {"window": 14, "column": "Close"},
        # Add other indicators if your strategy needs them directly for signals
    },
    # Add ETH, SOL configs if you adapt this script for them
}


# --- Step 3: Configuration for the Hourly Strategy ---
GDRIVE_INDICATORS_PATH = "/content/drive/MyDrive/CryptoDataCCXT/with_indicators/"
PAIR_SYMBOL = "BTC_USDT"
TIMEFRAME = "1h" # Change to "15m" to adapt for 15-minute data
FILE_TO_PROCESS = f"{PAIR_SYMBOL}_{TIMEFRAME}_with_indicators.csv"
full_file_path = os.path.join(GDRIVE_INDICATORS_PATH, FILE_TO_PROCESS)

BASE_COIN_SYMBOL = PAIR_SYMBOL.split('_')[0] # e.g., "BTC"

# Dynamically get indicator column names based on master_indicator_configs for BTC
# This makes the script more robust if you change parameters in your config
try:
    btc_config = master_indicator_configs[BASE_COIN_SYMBOL]
    FAST_EMA_COL = f"{BASE_COIN_SYMBOL}_EMA_{btc_config['ema_short']['span']}"
    SLOW_EMA_COL = f"{BASE_COIN_SYMBOL}_EMA_{btc_config['ema_long']['span']}"
    RSI_COL = f"RSI_{btc_config['rsi']['window']}" # Assuming your add_rsi names it like RSI_WINDOW
except KeyError as e:
    print(f"ERROR: Key {e} not found in master_indicator_configs for {BASE_COIN_SYMBOL}.")
    print("Please ensure master_indicator_configs is correctly defined and includes ema_short, ema_long, and rsi for BTC.")
    # Fallback to common defaults if config is missing for some reason (not recommended for production)
    # FAST_EMA_COL = f"{BASE_COIN_SYMBOL}_EMA_12"
    # SLOW_EMA_COL = f"{BASE_COIN_SYMBOL}_EMA_26"
    # RSI_COL = f"RSI_14"
    # print(f"Warning: Using default column names: {FAST_EMA_COL}, {SLOW_EMA_COL}, {RSI_COL}")
    raise # Stop execution if config is missing

RSI_THRESHOLD = 50 # RSI must be above this for a buy signal

print(f"\n--- Developing Hourly Strategy for: {full_file_path} ---")
print(f"Using Fast EMA: {FAST_EMA_COL}, Slow EMA: {SLOW_EMA_COL}, RSI: {RSI_COL} (threshold > {RSI_THRESHOLD})")

if not os.path.exists(full_file_path):
    print(f"ERROR: File not found at {full_file_path}. Cannot proceed.")
else:
    try:
        df = pd.read_csv(full_file_path, index_col='datetime_utc', parse_dates=True)
        print(f"\nSuccessfully loaded data. Shape: {df.shape}")
        print("First 3 rows of loaded data:")
        pd.set_option('display.max_columns', None)
        print(df.head(3).T)
        pd.reset_option('display.max_columns')


        # --- Step 4: Ensure Required Indicator Columns Exist ---
        required_indicator_cols = [FAST_EMA_COL, SLOW_EMA_COL, RSI_COL, 'Close'] # Need 'Close' for context
        missing_cols = [col for col in required_indicator_cols if col not in df.columns]
        if missing_cols:
            print(f"\nERROR: DataFrame is missing one or more required indicator columns for the strategy: {missing_cols}")
            print(f"Available columns: {df.columns.tolist()}")
            # Terminate or handle error appropriately
            raise ValueError("Missing required indicator columns for strategy.")
        else:
            print(f"\nAll required indicator columns found: {required_indicator_cols}")

            # --- Step 5: Generate Trading Signals ---
            df['signal'] = 0 # 0: Hold, 1: Buy (Enter Long), -1: Sell (Exit Long)

            # Condition for Fast EMA > Slow EMA
            df['fast_ema_gt_slow_ema'] = df[FAST_EMA_COL] > df[SLOW_EMA_COL]
            # Condition for RSI > Threshold
            df['rsi_gt_threshold'] = df[RSI_COL] > RSI_THRESHOLD

            # Buy Signal: Fast EMA just crossed above Slow EMA AND RSI is above threshold
            df['buy_trigger'] = (df['fast_ema_gt_slow_ema'] &
                                 (df['fast_ema_gt_slow_ema'].shift(1) == False) &
                                 df['rsi_gt_threshold'])
            df.loc[df['buy_trigger'], 'signal'] = 1

            # Sell Signal (Exit Long): Fast EMA just crossed below Slow EMA
            df['sell_trigger'] = ((df['fast_ema_gt_slow_ema'] == False) &
                                  (df['fast_ema_gt_slow_ema'].shift(1)))
            df.loc[df['sell_trigger'], 'signal'] = -1

            # Clean up temporary columns
            df.drop(columns=['fast_ema_gt_slow_ema', 'rsi_gt_threshold', 'buy_trigger', 'sell_trigger'], inplace=True)

            # --- Step 6: Display Signal Results ---
            print("\n--- Hourly Strategy Signals Generated ---")
            signals_generated_df = df[df['signal'] != 0]
            print(f"Number of buy signals: {len(df[df['signal'] == 1])}")
            print(f"Number of sell signals (exit long): {len(df[df['signal'] == -1])}")

            if not signals_generated_df.empty:
                print("\nFirst 10 signals (Buy=1, Sell=-1):")
                print(signals_generated_df[['Close', FAST_EMA_COL, SLOW_EMA_COL, RSI_COL, 'signal']].head(10))
                print("\nLast 10 signals:")
                print(signals_generated_df[['Close', FAST_EMA_COL, SLOW_EMA_COL, RSI_COL, 'signal']].tail(10))
            else:
                print("No trade signals were generated by this strategy.")

            # For a full backtest, you would now simulate trades based on these signals,
            # applying position sizing, commissions, stop-losses, and take-profits.
            print("\nReminder: This script generates signals. Full backtesting would simulate trades and P&L.")
            print("Consider adding ATR-based stop-losses and take-profits for a more complete hourly strategy.")

    except ValueError as ve:
        print(ve) # Print specific ValueError from column check
    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()

print("\n--- Hourly strategy development script segment complete ---")

In [ ]:
# PASTE YOUR master_indicator_configs DICTIONARY HERE
master_indicator_configs = {
    "BTC": {
        "sma_short":    {"window": 10, "column": "Close"},
        "sma_long":     {"window": 50, "column": "Close"},
        "ema_short":    {"span": 12, "column": "Close"},
        "ema_long":     {"span": 26, "column": "Close"},
        "rsi":          {"window": 14, "column": "Close"},
        "bollinger":    {"window": 20, "num_std_dev": 2, "column": "Close"},
        "macd":         {"short_window": 12, "long_window": 26, "signal_window": 9, "column": "Close"},
        "atr":          {"window": 14}, # Assumes High, Low, Close columns
        "stochastic":   {"k_window": 14, "d_window": 3}, # Assumes High, Low, Close
        "mfi":          {"window": 14}, # Assumes High, Low, Close, Volume
        "roc":          {"window": 10, "column": "Close"},
        "keltner":      {"ema_window": 20, "atr_window": 10, "atr_multiplier": 2}, # Assumes High, Low, Close
        "volume_sma":   {"window": 20}, # Assumes Volume column
        "support_resistance": {"window": 15} # Assumes High, Low columns
    },
    "ETH": { # Ensure ETH config is complete if you process ETH
        "sma_short":    {"window": 12, "column": "Close"},
        "sma_long":     {"window": 55, "column": "Close"},
        "ema_short":    {"span": 10, "column": "Close"},
        "ema_long":     {"span": 30, "column": "Close"},
        "rsi":          {"window": 14, "column": "Close"},
        "bollinger":    {"window": 20, "num_std_dev": 2.1, "column": "Close"},
        "macd":         {"short_window": 12, "long_window": 26, "signal_window": 9, "column": "Close"},
        "atr":          {"window": 10},
        "stochastic":   {"k_window": 10, "d_window": 3},
        "mfi":          {"window": 12},
        "roc":          {"window": 12, "column": "Close"},
        "keltner":      {"ema_window": 22, "atr_window": 11, "atr_multiplier": 2},
        "volume_sma":   {"window": 25},
        "support_resistance": {"window": 20}
    },
    "SOL": { # Ensure SOL config is complete
        "sma_short":    {"window": 9, "column": "Close"},
        "sma_long":     {"window": 45, "column": "Close"},
        "ema_short":    {"span": 12, "column": "Close"},
        "ema_long":     {"span": 26, "column": "Close"},
        "rsi":          {"window": 14, "column": "Close"},
        "bollinger":    {"window": 20, "num_std_dev": 2, "column": "Close"},
        "macd":         {"short_window": 12, "long_window": 26, "signal_window": 9, "column": "Close"},
        "atr":          {"window": 14},
        "stochastic":   {"k_window": 14, "d_window": 3},
        "mfi":          {"window": 14},
        "roc":          {"window": 10, "column": "Close"},
        "keltner":      {"ema_window": 20, "atr_window": 10, "atr_multiplier": 1.8},
        "volume_sma":   {"window": 20},
        "support_resistance": {"window": 15}
    }
}

In [ ]:
# PASTE ALL 13 add_... INDICATOR FUNCTION DEFINITIONS HERE
# (add_sma, add_ema, add_rsi, add_bollinger_bands, add_macd, add_atr,
#  add_stochastic_oscillator, add_mfi, add_roc, add_keltner_channels,
#  add_volume_sma, add_basic_support_resistance)
# ... ensure numpy is imported within them or globally (import numpy as np)
import numpy as np # Make sure numpy is imported if your functions use it directly
import pandas as pd # Should already be imported, but good practice

def add_sma(df, window=20, column='Close', new_column_name=None):
    if new_column_name is None: new_column_name = f'SMA_{window}'
    df[new_column_name] = df[column].rolling(window=window, min_periods=1).mean()
    return df

def add_ema(df, span=20, column='Close', new_column_name=None):
    if new_column_name is None: new_column_name = f'EMA_{span}'
    df[new_column_name] = df[column].ewm(span=span, adjust=False, min_periods=1).mean()
    return df

def add_rsi(df, window=14, column='Close', new_column_name=None):
    if new_column_name is None: new_column_name = f'RSI_{window}'
    delta = df[column].diff(1)
    gain = delta.where(delta > 0, 0).fillna(0)
    loss = -delta.where(delta < 0, 0).fillna(0)
    avg_gain = gain.ewm(alpha=1/window, adjust=False, min_periods=1).mean()
    avg_loss = loss.ewm(alpha=1/window, adjust=False, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rs = rs.replace([np.inf, -np.inf], np.nan).fillna(0)
    rsi = 100 - (100 / (1 + rs))
    rsi = np.where(avg_loss == 0, 100, rsi)
    rsi = np.where(avg_gain == 0, 0, rsi) # Handles all losses case
    df[new_column_name] = rsi
    return df

def add_bollinger_bands(df, window=20, num_std_dev=2, column='Close', sma_col_name=None, upper_band_col_name=None, lower_band_col_name=None):
    if sma_col_name is None: sma_col_name = f'BB_SMA_{window}'
    if upper_band_col_name is None: upper_band_col_name = f'BB_Upper_{window}'
    if lower_band_col_name is None: lower_band_col_name = f'BB_Lower_{window}'
    df[sma_col_name] = df[column].rolling(window=window, min_periods=1).mean()
    std_dev = df[column].rolling(window=window, min_periods=1).std(ddof=0) # ddof=0 for population std dev if preferred for BB
    df[upper_band_col_name] = df[sma_col_name] + (std_dev * num_std_dev)
    df[lower_band_col_name] = df[sma_col_name] - (std_dev * num_std_dev)
    return df

def add_macd(df, short_window=12, long_window=26, signal_window=9, column='Close', macd_line_col_name=None, signal_line_col_name=None, macd_hist_col_name=None):
    if macd_line_col_name is None: macd_line_col_name = f'MACD_{short_window}_{long_window}'
    if signal_line_col_name is None: signal_line_col_name = f'MACD_Signal_{signal_window}'
    if macd_hist_col_name is None: macd_hist_col_name = f'MACD_Hist_{signal_window}'
    ema_short = df[column].ewm(span=short_window, adjust=False, min_periods=1).mean()
    ema_long = df[column].ewm(span=long_window, adjust=False, min_periods=1).mean()
    df[macd_line_col_name] = ema_short - ema_long
    df[signal_line_col_name] = df[macd_line_col_name].ewm(span=signal_window, adjust=False, min_periods=1).mean()
    df[macd_hist_col_name] = df[macd_line_col_name] - df[signal_line_col_name]
    return df

def add_atr(df, window=14, high_col='High', low_col='Low', close_col='Close', atr_col_name=None):
    if atr_col_name is None: atr_col_name = f'ATR_{window}'
    # Ensure columns exist before trying to use them
    if not all(col in df.columns for col in [high_col, low_col, close_col]):
        print(f"ATR calc: Missing one or more required columns: {high_col}, {low_col}, {close_col}")
        df[atr_col_name] = np.nan
        return df

    df['_prev_close'] = df[close_col].shift(1)
    df['_h_minus_l'] = df[high_col] - df[low_col]
    df['_h_minus_pc'] = abs(df[high_col] - df['_prev_close'])
    df['_l_minus_pc'] = abs(df[low_col] - df['_prev_close'])
    # Ensure NaNs from shift(1) don't break max() if it's the first row, fill with 0 or handle
    df['_true_range'] = df[['_h_minus_l', '_h_minus_pc', '_l_minus_pc']].max(axis=1)
    df[atr_col_name] = df['_true_range'].ewm(alpha=1/window, adjust=False, min_periods=1).mean()
    df.drop(columns=['_prev_close', '_h_minus_l', '_h_minus_pc', '_l_minus_pc', '_true_range'], inplace=True, errors='ignore')
    return df

def add_stochastic_oscillator(df, k_window=14, d_window=3,
                              high_col='High', low_col='Low', close_col='Close',
                              stoch_k_col_name=None, stoch_d_col_name=None):
    if stoch_k_col_name is None: stoch_k_col_name = f'Stoch_%K_{k_window}'
    if stoch_d_col_name is None: stoch_d_col_name = f'Stoch_%D_{d_window}'
    if not all(col in df.columns for col in [high_col, low_col, close_col]):
        print(f"Stochastic calc: Missing one or more required columns: {high_col}, {low_col}, {close_col}")
        df[stoch_k_col_name] = np.nan
        df[stoch_d_col_name] = np.nan
        return df

    lowest_low = df[low_col].rolling(window=k_window, min_periods=1).min()
    highest_high = df[high_col].rolling(window=k_window, min_periods=1).max()
    denominator = highest_high - lowest_low
    df[stoch_k_col_name] = ((df[close_col] - lowest_low) / (denominator + 1e-9)) * 100 # Epsilon for stability
    df[stoch_k_col_name] = df[stoch_k_col_name].replace([np.inf, -np.inf], np.nan)
    df[stoch_k_col_name] = df[stoch_k_col_name].ffill().bfill() # Address NaNs before clip
    df[stoch_k_col_name] = df[stoch_k_col_name].clip(0, 100)
    df[stoch_d_col_name] = df[stoch_k_col_name].rolling(window=d_window, min_periods=1).mean()
    return df

def add_mfi(df, window=14, high_col='High', low_col='Low', close_col='Close', volume_col='Volume', mfi_col_name=None):
    if mfi_col_name is None: mfi_col_name = f'MFI_{window}'
    if not all(col in df.columns for col in [high_col, low_col, close_col, volume_col]):
        print(f"MFI calc: Missing one or more required columns: {high_col}, {low_col}, {close_col}, {volume_col}")
        df[mfi_col_name] = np.nan
        return df

    df['_typical_price'] = (df[high_col] + df[low_col] + df[close_col]) / 3
    df['_raw_money_flow'] = df['_typical_price'] * df[volume_col]
    price_diff = df['_typical_price'].diff(1)
    df['_positive_money_flow'] = df['_raw_money_flow'].where(price_diff > 0, 0).fillna(0)
    df['_negative_money_flow'] = df['_raw_money_flow'].where(price_diff < 0, 0).fillna(0)
    positive_mf_sum = df['_positive_money_flow'].rolling(window=window, min_periods=1).sum()
    negative_mf_sum = df['_negative_money_flow'].rolling(window=window, min_periods=1).sum()
    money_flow_ratio = positive_mf_sum / (negative_mf_sum + 1e-9)
    mfi = 100 - (100 / (1 + money_flow_ratio))
    mfi = mfi.replace([np.inf, -np.inf], 100) # If negative_mf_sum was 0, MFR is inf, MFI is 100
    mfi = np.where((positive_mf_sum == 0) & (negative_mf_sum == 0), 50, mfi) # Neutral if no flow
    mfi = np.where((positive_mf_sum > 0) & (negative_mf_sum == 0), 100, mfi) # All positive flow
    mfi = np.where((positive_mf_sum == 0) & (negative_mf_sum > 0), 0, mfi)   # All negative flow
    df[mfi_col_name] = mfi
    df.drop(columns=['_typical_price', '_raw_money_flow', '_positive_money_flow', '_negative_money_flow'], inplace=True, errors='ignore')
    return df

def add_roc(df, window=12, column='Close', new_column_name=None):
    if new_column_name is None: new_column_name = f'ROC_{window}'
    df[new_column_name] = df[column].pct_change(periods=window) * 100
    return df

def add_keltner_channels(df, ema_window=20, atr_window=10, atr_multiplier=2, high_col='High', low_col='Low', close_col='Close', middle_col_name=None, upper_col_name=None, lower_col_name=None):
    if middle_col_name is None: middle_col_name = f'KC_Middle_{ema_window}'
    if upper_col_name is None: upper_col_name = f'KC_Upper_{ema_window}'
    if lower_col_name is None: lower_col_name = f'KC_Lower_{ema_window}'

    if not all(col in df.columns for col in [high_col, low_col, close_col]):
        print(f"Keltner calc: Missing one or more required columns: {high_col}, {low_col}, {close_col}")
        df[middle_col_name] = np.nan
        df[upper_col_name] = np.nan
        df[lower_col_name] = np.nan
        return df

    df[middle_col_name] = df[close_col].ewm(span=ema_window, adjust=False, min_periods=1).mean()

    # Use a temporary ATR column name that is unlikely to clash
    temp_atr_col_for_kc = f'_temp_atr_kc_{atr_window}'
    # Make a copy for ATR calculation to avoid modifying df if add_atr drops columns from original
    df_for_atr = df[[high_col, low_col, close_col]].copy()
    df_for_atr = add_atr(df_for_atr, window=atr_window, high_col=high_col, low_col=low_col, close_col=close_col, atr_col_name=temp_atr_col_for_kc)

    if temp_atr_col_for_kc in df_for_atr.columns:
        df[upper_col_name] = df[middle_col_name] + (df_for_atr[temp_atr_col_for_kc] * atr_multiplier)
        df[lower_col_name] = df[middle_col_name] - (df_for_atr[temp_atr_col_for_kc] * atr_multiplier)
    else:
        print(f"Keltner calc: Failed to calculate ATR ({temp_atr_col_for_kc}). Setting bands to NaN.")
        df[upper_col_name] = np.nan
        df[lower_col_name] = np.nan
    return df

def add_volume_sma(df, window=20, volume_col='Volume', new_column_name=None):
    if new_column_name is None: new_column_name = f'Volume_SMA_{window}'
    if volume_col not in df.columns:
        print(f"Volume SMA calc: Missing volume column: {volume_col}")
        df[new_column_name] = np.nan
        return df
    df[new_column_name] = df[volume_col].rolling(window=window, min_periods=1).mean()
    return df

def add_basic_support_resistance(df, window=20, high_col='High', low_col='Low', support_col_name=None, resistance_col_name=None):
    if support_col_name is None: support_col_name = f'Support_{window}'
    if resistance_col_name is None: resistance_col_name = f'Resistance_{window}'
    if not all(col in df.columns for col in [high_col, low_col]):
        print(f"Support/Resistance calc: Missing one or more required columns: {high_col}, {low_col}")
        df[support_col_name] = np.nan
        df[resistance_col_name] = np.nan
        return df

    df[resistance_col_name] = df[high_col].rolling(window=window, min_periods=1).max().shift(1)
    df[support_col_name] = df[low_col].rolling(window=window, min_periods=1).min().shift(1)
    return df

In [ ]:
# Main script to generate indicators for 1h and 15m BTC data
import pandas as pd
import os
import numpy as np # Already imported with functions, but good practice here too
from google.colab import drive

# --- Step 1: Mount Google Drive ---
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    raise RuntimeError("Google Drive could not be mounted. Script aborted.")

# --- Step 2: Configuration ---
BASE_OHLCV_PATH = "/content/drive/MyDrive/CryptoDataCCXT/" # Path to original CCXT data (lowercase columns)
# Let's save to a new subfolder to keep things organized
OUTPUT_INDICATORS_PATH = "/content/drive/MyDrive/CryptoDataCCXT/with_indicators_h_m15_custom/"
if not os.path.exists(OUTPUT_INDICATORS_PATH):
    os.makedirs(OUTPUT_INDICATORS_PATH)
    print(f"Created output directory: {OUTPUT_INDICATORS_PATH}")

PAIR_SYMBOL_TO_PROCESS = "BTC_USDT"
TIMEFRAMES_TO_PROCESS = ["1h", "15m"]
DURATION_TAG = "4years" # From your base filenames

BASE_COIN_FOR_CONFIG = PAIR_SYMBOL_TO_PROCESS.split('_')[0] # e.g., "BTC"

# --- Step 3: Loop through timeframes and process ---
for timeframe in TIMEFRAMES_TO_PROCESS:
    input_filename = f"{PAIR_SYMBOL_TO_PROCESS}_{timeframe}_{DURATION_TAG}.csv"
    full_input_path = os.path.join(BASE_OHLCV_PATH, input_filename)

    # Define a clear output filename
    output_filename = f"{PAIR_SYMBOL_TO_PROCESS}_{timeframe}_all_custom_indicators.csv"
    full_output_path = os.path.join(OUTPUT_INDICATORS_PATH, output_filename)

    print(f"\n\n======================================================================")
    print(f"Processing: {full_input_path}")
    print(f"Output will be: {full_output_path}")
    print(f"======================================================================")

    if not os.path.exists(full_input_path):
        print(f"ERROR: Input file not found: {full_input_path}. Skipping.")
        continue

    try:
        # Load the base OHLCV CSV file
        df = pd.read_csv(full_input_path)
        print(f"  Successfully loaded base data. Shape: {df.shape}")

        # Standardize Datetime Index
        if 'datetime_utc' in df.columns:
            df['datetime_utc'] = pd.to_datetime(df['datetime_utc'])
            df.set_index('datetime_utc', inplace=True)
            print("  Used 'datetime_utc' column as DatetimeIndex.")
        elif 'timestamp' in df.columns and not isinstance(df.index, pd.DatetimeIndex):
            df['datetime_utc'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
            df.set_index('datetime_utc', inplace=True)
            print("  Used 'timestamp' (ms) column to create and set DatetimeIndex 'datetime_utc'.")
        else:
            print("  Warning: No 'datetime_utc' or 'timestamp' column found to set as DatetimeIndex.")
            # If index is not datetime, many indicators might fail or behave unexpectedly.
            # Depending on your base files, you might need to handle this.
            # For now, we assume one of them exists.

        # Ensure OHLCV columns are uppercase for indicator functions
        rename_map = {
            'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'
        }
        # Only rename if lowercase versions exist and uppercase don't already
        cols_to_rename_now = {}
        for Lc, Uc in rename_map.items():
            if Lc in df.columns and Uc not in df.columns:
                cols_to_rename_now[Lc] = Uc
            elif Lc in df.columns and Uc in df.columns and Lc != Uc: # Both exist, prefer Uc, warn
                print(f"  Warning: Both '{Lc}' and '{Uc}' exist. Using '{Uc}'.")


        if cols_to_rename_now:
            df.rename(columns=cols_to_rename_now, inplace=True)
            print(f"  Renamed base OHLCV columns to uppercase: {list(cols_to_rename_now.values())}")

        # Check for essential columns after rename
        essential_cols = ['Open', 'High', 'Low', 'Close', 'Volume'] # Volume might be optional for some indicators
        missing_essentials = [col for col in essential_cols if col not in df.columns]
        if any(col in missing_essentials for col in ['Open', 'High', 'Low', 'Close']): # Core OHLC are critical
            print(f"  ERROR: Missing critical OHLC columns after renaming: {missing_essentials}. Skipping indicator calculation for this file.")
            continue
        if 'Volume' not in df.columns:
            print("  Warning: 'Volume' column not found. Volume-based indicators will have NaNs.")


        # --- Apply All Configured Indicators for BTC ---
        if BASE_COIN_FOR_CONFIG in master_indicator_configs:
            current_config = master_indicator_configs[BASE_COIN_FOR_CONFIG]
            print(f"\n  Applying all configured indicators for {BASE_COIN_FOR_CONFIG} on {timeframe} data...")

            # Make a copy to add indicators to, preserving original loaded df for this iteration
            df_with_indicators = df.copy()

            # Moving Averages
            if "sma_short" in current_config:
                params = current_config["sma_short"]
                df_with_indicators = add_sma(df_with_indicators, **params, new_column_name=f'{BASE_COIN_FOR_CONFIG}_SMA_{params.get("window")}')
            if "sma_long" in current_config:
                params = current_config["sma_long"]
                df_with_indicators = add_sma(df_with_indicators, **params, new_column_name=f'{BASE_COIN_FOR_CONFIG}_SMA_{params.get("window")}')
            if "ema_short" in current_config:
                params = current_config["ema_short"]
                df_with_indicators = add_ema(df_with_indicators, **params, new_column_name=f'{BASE_COIN_FOR_CONFIG}_EMA_{params.get("span")}')
            if "ema_long" in current_config:
                params = current_config["ema_long"]
                df_with_indicators = add_ema(df_with_indicators, **params, new_column_name=f'{BASE_COIN_FOR_CONFIG}_EMA_{params.get("span")}')

            # Other Non-Volume Dependent Indicators
            if "rsi" in current_config: df_with_indicators = add_rsi(df_with_indicators, **current_config["rsi"])
            if "bollinger" in current_config: df_with_indicators = add_bollinger_bands(df_with_indicators, **current_config["bollinger"])
            if "macd" in current_config: df_with_indicators = add_macd(df_with_indicators, **current_config["macd"])
            if "atr" in current_config: df_with_indicators = add_atr(df_with_indicators, **current_config["atr"])
            if "stochastic" in current_config: df_with_indicators = add_stochastic_oscillator(df_with_indicators, **current_config["stochastic"])
            if "roc" in current_config: df_with_indicators = add_roc(df_with_indicators, **current_config["roc"])
            if "keltner" in current_config: df_with_indicators = add_keltner_channels(df_with_indicators, **current_config["keltner"])
            if "support_resistance" in current_config: df_with_indicators = add_basic_support_resistance(df_with_indicators, **current_config["support_resistance"])

            # Indicators REQUIRING Volume
            if 'Volume' in df_with_indicators.columns:
                print(f"  DEBUG: 'Volume' column available. Proceeding with volume-based indicators.")
                if "mfi" in current_config:
                    df_with_indicators = add_mfi(df_with_indicators, **current_config["mfi"])
                if "volume_sma" in current_config:
                    df_with_indicators = add_volume_sma(df_with_indicators, **current_config["volume_sma"])
            else:
                print(f"  WARNING: 'Volume' column NOT found in DataFrame. Skipping MFI and Volume SMA, or they will have NaNs if functions add NaN columns.")
                # Ensure NaN columns are added if functions don't handle missing input columns by adding NaNs themselves
                if "mfi" in current_config: df_with_indicators[f'MFI_{current_config["mfi"].get("window",14)}'] = np.nan
                if "volume_sma" in current_config: df_with_indicators[f'Volume_SMA_{current_config["volume_sma"].get("window",20)}'] = np.nan

            print("  All configured indicators applied.")

            # --- Save the DataFrame with Indicators ---
            df_with_indicators.to_csv(full_output_path)
            print(f"  Successfully saved DataFrame with indicators to: {full_output_path}")
            print(f"  New file shape: {df_with_indicators.shape}")
            print(f"  New file columns (first 10): {df_with_indicators.columns.tolist()[:10]}")

        else:
            print(f"  ERROR: No configuration found for '{BASE_COIN_FOR_CONFIG}' in master_indicator_configs. Cannot apply indicators.")

    except FileNotFoundError:
        # This should be caught by the earlier check, but as a safeguard
        print(f"  ERROR: File not found during processing: {full_input_path}")
    except Exception as e:
        print(f"  An error occurred while processing {full_input_path}: {e}")
        import traceback
        traceback.print_exc()

print("\n\n--- All 1h and 15m BTC Indicator Generation Complete ---")

In [ ]:
import pandas as pd
import os
from google.colab import drive # For Google Drive access

# Ensure backtesting library is installed
# !pip install backtesting

from backtesting import Backtest, Strategy
from backtesting.lib import crossover # For detecting crossovers

# --- Step 1: Mount Google Drive ---
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    raise RuntimeError("Google Drive could not be mounted. Script aborted.")

# --- Step 2: Define Your Strategy Class (Generates Signals Internally) ---
# This class structure is the same, the position_size_pct will be passed from bt.run()
class EmaRsiAtrDynamicSizeStrategy(Strategy):
    # --- Strategy Parameters (will be set via bt.run()) ---
    atr_multiplier_sl = 1.5
    atr_multiplier_tp = 3.0
    rsi_buy_threshold = 50
    position_size_pct = 0.10 # Default if not passed, but we will pass it

    fast_ema_col_name = 'DEFAULT_FAST_EMA'
    slow_ema_col_name = 'DEFAULT_SLOW_EMA'
    rsi_col_name = 'DEFAULT_RSI'
    atr_col_name = 'DEFAULT_ATR'

    def init(self):
        if self.fast_ema_col_name not in self.data.df.columns: raise ValueError(f"Fast EMA column '{self.fast_ema_col_name}' not found. Cols: {self.data.df.columns.tolist()}")
        if self.slow_ema_col_name not in self.data.df.columns: raise ValueError(f"Slow EMA column '{self.slow_ema_col_name}' not found. Cols: {self.data.df.columns.tolist()}")
        if self.rsi_col_name not in self.data.df.columns: raise ValueError(f"RSI column '{self.rsi_col_name}' not found. Cols: {self.data.df.columns.tolist()}")
        if self.atr_col_name not in self.data.df.columns: raise ValueError(f"ATR column '{self.atr_col_name}' not found. Cols: {self.data.df.columns.tolist()}")

        self.fast_ema = self.I(lambda x: x, self.data.df[self.fast_ema_col_name], name="FastEMA")
        self.slow_ema = self.I(lambda x: x, self.data.df[self.slow_ema_col_name], name="SlowEMA")
        self.rsi = self.I(lambda x: x, self.data.df[self.rsi_col_name], name="RSI")
        self.atr = self.I(lambda x: x, self.data.df[self.atr_col_name], name="ATR")

    def next(self):
        if len(self.data.Close) < 2: # Ensure there's at least one previous bar for signal calculation
            return

        # Signals are based on the PREVIOUS bar's close (i-1, or self.data.Close[-2] if current bar is -1)
        # Trade execution at the CURRENT bar's open (i, or self.data.Open[-1] if current bar is -1)

        # Using -2 to access data from the bar that just closed, decision for open of bar -1 (current processing bar)
        # backtesting.py calls next() for each bar. self.data.Close[-1] is the current bar's close for which signal forms.
        # Trade is then for next bar.
        # crossover() uses current and previous values internally.

        price_for_sl_tp_calc = self.data.Close[-1] # Price of the bar where signal is confirmed (its close)
        current_atr_for_sl_tp = self.atr[-1]      # ATR of the bar where signal is confirmed
        entry_price_next_bar_open = self.data.Open[-1] # This is actually current bar's open
                                                     # In backtesting.py, self.buy() places order for *next* bar.
                                                     # So, decision based on Close[-1], ATR[-1]. Order for Open[current_bar_idx+1].
                                                     # For sizing, we can use Close[-1] as proxy for next open.

        # --- Entry Conditions (Long Only) ---
        fast_ema_crossed_above_slow_ema = crossover(self.fast_ema, self.slow_ema)
        rsi_is_bullish = self.rsi[-1] > self.rsi_buy_threshold

        if fast_ema_crossed_above_slow_ema and rsi_is_bullish and not self.position:
            if pd.notna(current_atr_for_sl_tp) and current_atr_for_sl_tp > 0:

                # For SL/TP, anchor to the estimated entry price (next bar's open, proxied by current close)
                sl_price = price_for_sl_tp_calc - (current_atr_for_sl_tp * self.atr_multiplier_sl)
                tp_price = price_for_sl_tp_calc + (current_atr_for_sl_tp * self.atr_multiplier_tp)

                # Calculate size based on equity and estimated entry price
                # The trade will actually happen at the next bar's open.
                # Using current close as proxy for next open for sizing.
                # self.equity is the current cash + value of open positions.
                # If not in position, self.equity is effectively cash.
                cash_to_invest = self.equity * self.position_size_pct
                size_in_units = cash_to_invest / price_for_sl_tp_calc # Using current close as proxy for next open

                if size_in_units > 1e-8: # Avoid extremely small or zero size
                    self.buy(size=size_in_units, sl=sl_price, tp=tp_price)
            else:
                if not (pd.notna(current_atr_for_sl_tp) and current_atr_for_sl_tp > 0):
                     print(f"Warning: ATR is invalid ({current_atr_for_sl_tp:.2f}) on {self.data.index[-1]}. Cannot set SL/TP. Skipping buy signal.")

        # --- Exit Conditions (Long Only) based on EMA cross ---
        fast_ema_crossed_below_slow_ema = crossover(self.slow_ema, self.fast_ema)

        if fast_ema_crossed_below_slow_ema and self.position:
            self.position.close()

# --- Step 3: Master Indicator Configuration (Placeholder - ENSURE THIS IS DEFINED IN YOUR NOTEBOOK) ---
# This dictionary is crucial for dynamically getting the correct column names.
master_indicator_configs = {
    "BTC": {
        "ema_short": {"span": 12, "column": "Close"},
        "ema_long":  {"span": 26, "column": "Close"},
        "rsi":       {"window": 14, "column": "Close"},
        "atr":       {"window": 14},
    },
    "ETH": {
        "ema_short": {"span": 12, "column": "Close"}, "ema_long":  {"span": 26, "column": "Close"},
        "rsi":       {"window": 14, "column": "Close"}, "atr":       {"window": 14},
    },
    "SOL": {
        "ema_short": {"span": 12, "column": "Close"}, "ema_long":  {"span": 26, "column": "Close"},
        "rsi":       {"window": 14, "column": "Close"}, "atr":       {"window": 14},
    }
}
# --- End of Master Indicator Configuration Placeholder ---


# --- Step 4: Main Configuration & Data Loading ---
GDRIVE_INDICATORS_PATH = "/content/drive/MyDrive/CryptoDataCCXT/with_indicators_h_m15_custom/"

PAIR_SYMBOL = "BTC_USDT"
TIMEFRAME = "1h" # Keeping 1h as per your last output. Change to "15m" if needed.

FILENAME_SUFFIX = "all_custom_indicators.csv"
file_to_backtest = f"{PAIR_SYMBOL}_{TIMEFRAME}_{FILENAME_SUFFIX}"
full_file_path = os.path.join(GDRIVE_INDICATORS_PATH, file_to_backtest)

BASE_COIN_SYMBOL = PAIR_SYMBOL.split('_')[0]

try:
    coin_config = master_indicator_configs[BASE_COIN_SYMBOL]
    if 'ema_short' not in coin_config or 'span' not in coin_config['ema_short']: raise KeyError("Config missing 'ema_short' or its 'span'")
    if 'ema_long' not in coin_config or 'span' not in coin_config['ema_long']: raise KeyError("Config missing 'ema_long' or its 'span'")
    if 'rsi' not in coin_config or 'window' not in coin_config['rsi']: raise KeyError("Config missing 'rsi' or its 'window'")
    if 'atr' not in coin_config or 'window' not in coin_config['atr']: raise KeyError("Config missing 'atr' or its 'window'")

    ACTUAL_FAST_EMA_COL_NAME = f"{BASE_COIN_SYMBOL}_EMA_{coin_config['ema_short']['span']}"
    ACTUAL_SLOW_EMA_COL_NAME = f"{BASE_COIN_SYMBOL}_EMA_{coin_config['ema_long']['span']}"
    ACTUAL_RSI_COL_NAME = f"RSI_{coin_config['rsi']['window']}"
    ACTUAL_ATR_COL_NAME = f"ATR_{coin_config['atr']['window']}"
except KeyError as e:
    print(f"CRITICAL ERROR with master_indicator_configs for {BASE_COIN_SYMBOL}: {e}.")
    raise

INITIAL_CAPITAL = 100000.0
COMMISSION_RATE_DECIMAL = 0.001
ATR_SL_MULTIPLIER_PARAM = 1.5
ATR_TP_MULTIPLIER_PARAM = 3.0
RSI_BUY_LEVEL_PARAM = 50
# --- KEY CHANGE: Reduce Position Size ---
POSITION_SIZE_PERCENT_PARAM = 0.10 # Using 10% of equity per trade

print(f"\n--- Backtesting Strategy for: {full_file_path} using backtesting.py ---")
print(f"Strategy uses: Fast EMA ({ACTUAL_FAST_EMA_COL_NAME}), Slow EMA ({ACTUAL_SLOW_EMA_COL_NAME}), RSI ({ACTUAL_RSI_COL_NAME}), ATR ({ACTUAL_ATR_COL_NAME})")
print(f"Position Size per trade: {POSITION_SIZE_PERCENT_PARAM*100}% of current equity.")

if not os.path.exists(full_file_path):
    print(f"ERROR: File not found at {full_file_path}. Cannot proceed.")
else:
    try:
        df_loaded = pd.read_csv(full_file_path, index_col='datetime_utc', parse_dates=True)
        print(f"Successfully loaded data. Shape: {df_loaded.shape}. Data from {df_loaded.index.min()} to {df_loaded.index.max()}")

        if df_loaded.empty:
            print("ERROR: Loaded DataFrame is empty.")
        else:
            required_ohlc_cols = ['Open', 'High', 'Low', 'Close']
            required_strategy_indicator_cols = [ACTUAL_FAST_EMA_COL_NAME, ACTUAL_SLOW_EMA_COL_NAME, ACTUAL_RSI_COL_NAME, ACTUAL_ATR_COL_NAME]
            all_needed_cols = required_ohlc_cols + required_strategy_indicator_cols
            missing_cols_in_df = [col for col in all_needed_cols if col not in df_loaded.columns]

            if missing_cols_in_df:
                 raise ValueError(f"DataFrame is missing one or more required columns for the strategy: {missing_cols_in_df}. Available columns: {df_loaded.columns.tolist()}")

            bt = Backtest(
                df_loaded, EmaRsiAtrDynamicSizeStrategy,
                cash=INITIAL_CAPITAL, commission=COMMISSION_RATE_DECIMAL,
                exclusive_orders=True, trade_on_close=False
            )

            stats = bt.run(
                atr_multiplier_sl=ATR_SL_MULTIPLIER_PARAM,
                atr_multiplier_tp=ATR_TP_MULTIPLIER_PARAM,
                rsi_buy_threshold=RSI_BUY_LEVEL_PARAM,
                position_size_pct=POSITION_SIZE_PERCENT_PARAM, # Pass the updated position size
                fast_ema_col_name=ACTUAL_FAST_EMA_COL_NAME,
                slow_ema_col_name=ACTUAL_SLOW_EMA_COL_NAME,
                rsi_col_name=ACTUAL_RSI_COL_NAME,
                atr_col_name=ACTUAL_ATR_COL_NAME
            )

            print("\n--- Backtest Performance Metrics (from backtesting.py) ---")
            print(stats)

            print(f"\nSelected Stats:")
            stat_keys_to_print = ['Return [%]', 'Max. Drawdown [%]', 'Win Rate [%]', '# Trades', 'Profit Factor', 'Sharpe Ratio', 'Sortino Ratio']
            for key in stat_keys_to_print:
                value = stats.get(key, 'N/A')
                if isinstance(value, float):
                    print(f"  {key}: {value:.2f}")
                else:
                    print(f"  {key}: {value}")

            plot_filename = os.path.join(GDRIVE_INDICATORS_PATH, f"{PAIR_SYMBOL}_{TIMEFRAME}_ema_rsi_atr_backtest_plot_pos_size_updated.html")
            try:
                bt.plot(filename=plot_filename, open_browser=False, superimpose=True)
                print(f"Backtest plot saved to: {plot_filename}")
            except Exception as plot_error:
                print(f"Could not generate plot: {plot_error}.")

    except ValueError as ve:
        print(f"ValueError during setup or backtest: {ve}")
    except Exception as e:
        print(f"An unexpected error occurred during the backtest: {e}")
        import traceback
        traceback.print_exc()

print("\n--- Backtesting script with backtesting.py complete ---")


In [ ]:
master_indicator_configs = {
    "BTC": {
        # Parameters for OptimizableEmaRsiAtrStrategy
        "ema_short_len":      {"default": 12, "optimize": range(9, 25, 3)},   # Test 9, 12, 15, 18, 21, 24
        "ema_long_len":       {"default": 26, "optimize": range(25, 55, 5)},  # Test 25, 30, ..., 50
        "rsi_len":            {"default": 14, "optimize": range(7, 22, 3)},    # Test 7, 10, 13, 16, 19
        "rsi_buy_threshold":  {"default": 50, "optimize": range(40, 61, 5)},   # Test 40, 45, 50, 55, 60
        "atr_len":            {"default": 14, "optimize": range(10, 21, 4)},   # Test 10, 14, 18
        "atr_multiplier_sl":  {"default": 1.5, "optimize": np.arange(1.0, 2.6, 0.5)}, # Test 1.0, 1.5, 2.0, 2.5
        "atr_multiplier_tp":  {"default": 3.0, "optimize": np.arange(2.0, 5.1, 1.0)}, # Test 2.0, 3.0, 4.0, 5.0
        "use_trend_filter":   {"default": True, "optimize": [True, False]},
        "long_sma_trend_len": {"default": 100, "optimize": range(50, 151, 25)}, # Test 50, 75, 100, 125, 150

        # You can keep your original indicator configs here too if used by other scripts
        # "sma_short":    {"window": 10, "column": "Close"},
        # ... etc.
    },
    "ETH": {
        # Define similar "param_name": {"default": X, "optimize": Y} for ETH
        "ema_short_len":      {"default": 12, "optimize": range(9, 25, 3)},
        "ema_long_len":       {"default": 26, "optimize": range(25, 55, 5)},
        # ... fill out for all optimizable parameters for ETH
        "rsi_len": {"default": 14, "optimize": [14]}, # Example: fix RSI window for ETH
        "rsi_buy_threshold": {"default": 50, "optimize": [50, 55]},
        "atr_len": {"default": 14, "optimize": [14]},
        "atr_multiplier_sl": {"default": 1.5, "optimize": [1.5, 2.0]},
        "atr_multiplier_tp": {"default": 3.0, "optimize": [3.0, 4.0]},
        "use_trend_filter":  {"default": True, "optimize": [True]},
        "long_sma_trend_len": {"default": 100, "optimize": [100, 120]},
    },
    "SOL": {
        # Define similar "param_name": {"default": X, "optimize": Y} for SOL
        "ema_short_len":      {"default": 12, "optimize": range(9, 25, 3)},
        "ema_long_len":       {"default": 26, "optimize": range(25, 55, 5)},
        # ... fill out for all optimizable parameters for SOL
        "rsi_len": {"default": 14, "optimize": [14]},
        "rsi_buy_threshold": {"default": 50, "optimize": [50]},
        "atr_len": {"default": 14, "optimize": [14]},
        "atr_multiplier_sl": {"default": 1.5, "optimize": [1.5]},
        "atr_multiplier_tp": {"default": 3.0, "optimize": [3.0]},
        "use_trend_filter":  {"default": True, "optimize": [True, False]},
        "long_sma_trend_len": {"default": 100, "optimize": [100]},
    }
}



In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import drive

# Ensure backtesting library is installed
# !pip install backtesting

from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# --- Helper functions for Indicators (same as before) ---
def ema_func(series, span):
    return series.ewm(span=span, adjust=False, min_periods=1).mean()

def sma_func(series, window):
    return series.rolling(window=window, min_periods=1).mean()

def rsi_func(series, window):
    if window == 0: return pd.Series(50., index=series.index) # Handle invalid window
    delta = series.diff(1)
    gain = delta.where(delta > 0, 0).fillna(0)
    loss = -delta.where(delta < 0, 0).fillna(0)
    # Ensure min_periods is at least 1, and ideally >= window for stable start
    min_p = max(1, int(window))
    avg_gain = gain.ewm(alpha=1/window, adjust=False, min_periods=min_p).mean()
    avg_loss = loss.ewm(alpha=1/window, adjust=False, min_periods=min_p).mean()
    rs = avg_gain / (avg_loss + 1e-9)
    rsi = 100 - (100 / (1 + rs))
    rsi = np.where(avg_loss < 1e-9, 100, rsi) # If avg_loss is effectively 0
    rsi = np.where(avg_gain < 1e-9, 0, rsi)   # If avg_gain is effectively 0
    return pd.Series(rsi, index=series.index).fillna(50) # Fill initial NaNs with neutral 50

def atr_func(high_series, low_series, close_series, window):
    if window == 0: return pd.Series(0., index=close_series.index) # Handle invalid window
    prev_close = close_series.shift(1)
    tr1 = abs(high_series - low_series)
    tr2 = abs(high_series - prev_close)
    tr3 = abs(low_series - prev_close)
    true_range = pd.DataFrame({'tr1': tr1, 'tr2': tr2, 'tr3': tr3}).max(axis=1)
    # Ensure min_periods is at least 1, and ideally >= window
    min_p = max(1, int(window))
    atr = true_range.ewm(alpha=1/window, adjust=False, min_periods=min_p).mean()
    return atr.fillna(0) # Fill initial NaNs with 0


# --- Step 1: Mount Google Drive ---
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    raise RuntimeError("Google Drive could not be mounted. Script aborted.")

# --- Step 2: Define Your Optimizable Strategy Class ---
class OptimizableEmaRsiAtrStrategy(Strategy):
    # Parameters will be set by bt.optimize() using values from master_indicator_configs
    # Class variables here act as defaults if a param is not optimized OR if running bt.run() directly
    ema_short_len = 12
    ema_long_len = 26
    rsi_len = 14
    rsi_buy_threshold = 50
    atr_len = 14
    atr_multiplier_sl = 1.5
    atr_multiplier_tp = 3.0
    use_trend_filter = True
    long_sma_trend_len = 100
    position_size_pct = 0.10 # Fixed for this optimization, can be optimized separately

    def init(self):
        # Ensure parameters are integers where appropriate (for window/span/length)
        self.p_ema_short_len = int(self.ema_short_len)
        self.p_ema_long_len = int(self.ema_long_len)
        self.p_rsi_len = int(self.rsi_len)
        self.p_atr_len = int(self.atr_len)
        self.p_long_sma_trend_len = int(self.long_sma_trend_len)

        self.fast_ema = self.I(ema_func, self.data.Close, span=self.p_ema_short_len, name="FastEMA")
        self.slow_ema = self.I(ema_func, self.data.Close, span=self.p_ema_long_len, name="SlowEMA")
        self.rsi_calc = self.I(rsi_func, self.data.Close, window=self.p_rsi_len, name="RSI")
        self.atr_calc = self.I(atr_func, self.data.High, self.data.Low, self.data.Close, window=self.p_atr_len, name="ATR")

        if self.use_trend_filter:
            self.long_sma_trend = self.I(sma_func, self.data.Close, window=self.p_long_sma_trend_len, name="TrendSMA")
        else:
            self.long_sma_trend = None

    def next(self):
        # Ensure enough data for all indicators based on their actual lengths
        min_bars_needed = max(self.p_ema_long_len, self.p_rsi_len, self.p_atr_len, (self.p_long_sma_trend_len if self.use_trend_filter else 0)) + 2
        if len(self.data.Close) < min_bars_needed:
            return

        price_for_sl_tp_calc = self.data.Close[-1]
        current_atr_for_sl_tp = self.atr_calc[-1]
        entry_price_this_bar_open = self.data.Open[-1] # This is for current bar's open if trade_on_close=False

        trend_is_up = True
        if self.use_trend_filter and self.long_sma_trend is not None:
            trend_is_up = self.data.Close[-1] > self.long_sma_trend[-1]

        fast_ema_crossed_above_slow_ema = crossover(self.fast_ema, self.slow_ema)
        rsi_is_bullish = self.rsi_calc[-1] > self.rsi_buy_threshold

        if trend_is_up and fast_ema_crossed_above_slow_ema and rsi_is_bullish and not self.position:
            if pd.notna(current_atr_for_sl_tp) and current_atr_for_sl_tp > 0 and pd.notna(price_for_sl_tp_calc) and price_for_sl_tp_calc > 0 : # Use price_for_sl_tp_calc also for sizing as proxy
                sl_price = price_for_sl_tp_calc - (current_atr_for_sl_tp * self.atr_multiplier_sl)
                tp_price = price_for_sl_tp_calc + (current_atr_for_sl_tp * self.atr_multiplier_tp)

                cash_to_invest = self.equity * self.position_size_pct
                size_in_units = cash_to_invest / price_for_sl_tp_calc # Using Close of signal bar as proxy for next open

                if size_in_units > 1e-8:
                    self.buy(size=size_in_units, sl=sl_price, tp=tp_price)
            else:
                # Minimal logging during optimization, can be enabled for debugging a single run
                # print(f"Trace: ATR or Price invalid. ATR: {current_atr_for_sl_tp}, Price: {price_for_sl_tp_calc}. Buy skipped on {self.data.index[-1]}.")
                pass


        fast_ema_crossed_below_slow_ema = crossover(self.slow_ema, self.fast_ema)
        if fast_ema_crossed_below_slow_ema and self.position:
            self.position.close()

# --- Step 3: Master Indicator Configuration ( ENSURE THIS IS DEFINED IN YOUR NOTEBOOK ) ---
# Paste your 'master_indicator_configs' dictionary here, structured for optimization.
# Example structure provided in the explanation above.
master_indicator_configs = {
    "BTC": {
        "ema_short_len":      {"default": 12, "optimize": range(10, 25, 3)}, # e.g., 10, 13, 16, 19, 22
        "ema_long_len":       {"default": 26, "optimize": range(25, 51, 5)},  # e.g., 25, 30, ... 50
        "rsi_len":            {"default": 14, "optimize": range(9, 20, 3)},   # e.g., 9, 12, 15, 18
        "rsi_buy_threshold":  {"default": 50, "optimize": [45, 50, 55]},     # Test specific values
        "atr_len":            {"default": 14, "optimize": range(10, 21, 5)},   # e.g., 10, 15, 20
        "atr_multiplier_sl":  {"default": 1.5, "optimize": np.arange(1.0, 2.6, 0.5)},
        "atr_multiplier_tp":  {"default": 3.0, "optimize": np.arange(2.0, 4.1, 0.5)},
        "use_trend_filter":   {"default": True, "optimize": [True, False]},
        "long_sma_trend_len": {"default": 100, "optimize": range(50, 151, 25)},
        # position_size_pct is fixed for this optimization run
    },
    # Define ETH, SOL similarly if you want to optimize for them later
}
# --- End of Master Indicator Configuration ---


# --- Step 4: Main Configuration & Data Loading for Optimization ---
GDRIVE_BASE_DATA_PATH = "/content/drive/MyDrive/CryptoDataCCXT/"
OUTPUT_OPTIMIZATION_PATH = "/content/drive/MyDrive/CryptoDataCCXT/optimization_results/"
if not os.path.exists(OUTPUT_OPTIMIZATION_PATH):
    os.makedirs(OUTPUT_OPTIMIZATION_PATH)

PAIR_SYMBOL = "BTC_USDT"
TIMEFRAME = "1h" # Change to "15m" if desired
DURATION_TAG = "4years"

file_to_backtest = f"{PAIR_SYMBOL}_{TIMEFRAME}_{DURATION_TAG}.csv" # Using BASE OHLCV file
full_file_path = os.path.join(GDRIVE_BASE_DATA_PATH, file_to_backtest)

INITIAL_CAPITAL = 100000.0
COMMISSION_RATE_DECIMAL = 0.001
FIXED_POSITION_SIZE_PCT = 0.10 # Keeping this fixed during this optimization round

print(f"\n--- Optimizing Strategy for: {full_file_path} using backtesting.py ---")
print(f"Fixed Position Size per trade: {FIXED_POSITION_SIZE_PCT*100}% of current equity.")

if not os.path.exists(full_file_path):
    print(f"ERROR: File not found at {full_file_path}. Cannot proceed.")
else:
    try:
        df_loaded_raw = pd.read_csv(full_file_path)

        if 'datetime_utc' in df_loaded_raw.columns:
            df_loaded_raw['datetime_utc'] = pd.to_datetime(df_loaded_raw['datetime_utc'])
            df_loaded_raw.set_index('datetime_utc', inplace=True)
        elif 'timestamp' in df_loaded_raw.columns:
            df_loaded_raw['datetime_utc'] = pd.to_datetime(df_loaded_raw['timestamp'], unit='ms', utc=True)
            df_loaded_raw.set_index('datetime_utc', inplace=True)
        else:
            raise ValueError("DataFrame must have 'datetime_utc' or 'timestamp' column for index.")

        rename_map = {'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}
        df_prepared = df_loaded_raw.rename(columns={k:v for k,v in rename_map.items() if k in df_loaded_raw.columns})

        required_ohlcv_cols = ['Open', 'High', 'Low', 'Close']
        if not all(col in df_prepared.columns for col in required_ohlcv_cols):
            raise ValueError(f"Prepared DataFrame must contain 'Open', 'High', 'Low', 'Close'. Found: {df_prepared.columns.tolist()}")
        if 'Volume' not in df_prepared.columns: # MFI uses Volume, ensure it's available if MFI part of strategy
            print("Warning: 'Volume' column not found in input data. Volume-based indicators will not work if strategy depends on them.")


        print(f"Successfully loaded and prepared data. Shape: {df_prepared.shape}. Data from {df_prepared.index.min()} to {df_prepared.index.max()}")

        if df_prepared.empty:
            print("ERROR: Loaded DataFrame is empty.")
        else:
            # --- Step 5: Prepare Optimization Grid from Master Config ---
            BASE_COIN_FOR_CONFIG = PAIR_SYMBOL.split('_')[0]
            if BASE_COIN_FOR_CONFIG not in master_indicator_configs:
                raise ValueError(f"No configuration found for {BASE_COIN_FOR_CONFIG} in master_indicator_configs.")

            coin_opt_config = master_indicator_configs[BASE_COIN_FOR_CONFIG]
            optimization_grid = {}
            strategy_fixed_params = {"position_size_pct": FIXED_POSITION_SIZE_PCT}

            # Parameters that the strategy class defines and that we want to optimize
            strategy_params_to_optimize = [
                "ema_short_len", "ema_long_len", "rsi_len", "rsi_buy_threshold",
                "atr_len", "atr_multiplier_sl", "atr_multiplier_tp",
                "use_trend_filter", "long_sma_trend_len"
            ]

            for param_name in strategy_params_to_optimize:
                if param_name in coin_opt_config and "optimize" in coin_opt_config[param_name]:
                    optimization_grid[param_name] = coin_opt_config[param_name]["optimize"]
                elif param_name in coin_opt_config and "default" in coin_opt_config[param_name]:
                    # If no "optimize" range, pass the "default" as a fixed parameter for this run
                    strategy_fixed_params[param_name] = coin_opt_config[param_name]["default"]
                    print(f"Using default value for {param_name}: {strategy_fixed_params[param_name]}")
                else:
                    # Use class default if not in config, bt.optimize will pass it if in grid
                    # or strategy class default will be used.
                    print(f"Parameter {param_name} not found in coin_opt_config or no 'optimize'/'default' key. Strategy class default will be used if not optimized.")


            if not optimization_grid:
                raise ValueError("Optimization grid is empty. Check 'master_indicator_configs' for 'optimize' keys.")

            print("\nOptimization Grid:")
            for k, v in optimization_grid.items():
                print(f"  {k}: {list(v) if isinstance(v, (range, type(np.arange(1)))) else v}")
            print("\nFixed Parameters for Strategy (not optimized in this run):")
            for k, v in strategy_fixed_params.items():
                print(f"  {k}: {v}")


            # --- Run Optimization ---
            bt = Backtest(
                df_prepared, OptimizableEmaRsiAtrStrategy,
                cash=INITIAL_CAPITAL, commission=COMMISSION_RATE_DECIMAL,
                exclusive_orders=True, trade_on_close=False
            )

            print("\nStarting optimization (this may take some time depending on ranges and data length)...")

            optimization_stats = bt.optimize(
                **optimization_grid, # Spread the optimization grid
                **strategy_fixed_params, # Spread fixed parameters
                maximize='SQN',
                # maximize='Sharpe Ratio',
                # maximize='Sortino Ratio',
                # maximize='Equity Final [$]',
                constraint=lambda params: params.ema_short_len < params.ema_long_len and \
                                           params.atr_multiplier_sl > 0 and \
                                           params.atr_multiplier_tp > params.atr_multiplier_sl, # More constraints
                max_tries=200 # Adjust as needed. If combinations are few, grid search is used.
                              # If many, random search up to max_tries.
            )

            print("\n--- Optimization Results (Best Strategy Stats) ---")
            print(optimization_stats)

            print("\n--- Best Strategy Parameters Found ---")
            best_params = optimization_stats._strategy # This is an instance of the strategy with best params
            print(f"  EMA Short Length: {best_params.ema_short_len}")
            print(f"  EMA Long Length: {best_params.ema_long_len}")
            print(f"  RSI Length: {best_params.rsi_len}")
            print(f"  RSI Buy Threshold: {best_params.rsi_buy_threshold}")
            print(f"  ATR Length: {best_params.atr_len}")
            print(f"  ATR SL Multiplier: {best_params.atr_multiplier_sl}")
            print(f"  ATR TP Multiplier: {best_params.atr_multiplier_tp}")
            print(f"  Use Trend Filter: {best_params.use_trend_filter}")
            print(f"  Long SMA Trend Length: {best_params.long_sma_trend_len}")

            # Plotting the best strategy
            plot_filename = os.path.join(OUTPUT_OPTIMIZATION_PATH, f"{PAIR_SYMBOL}_{TIMEFRAME}_optimized_plot.html")
            try:
                bt.plot(filename=plot_filename, open_browser=False, superimpose=True, plot_equity=True, plot_drawdown=True)
                print(f"Plot for BEST optimized strategy saved to: {plot_filename}")
            except Exception as plot_error:
                print(f"Could not generate plot for optimized strategy: {plot_error}.")

    except ValueError as ve:
        print(f"ValueError during setup or optimization: {ve}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()

print("\n--- Optimization script with backtesting.py complete ---")